In [ ]:
import logging
from importlib import reload
reload(logging)
logger = logging.getLogger(__name__)

In [ ]:
import pandas as pd
import json
import time
import random
import os
from ast import literal_eval
from json import JSONDecodeError
from pathlib import Path
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt
from pprint import pprint
from pathlib import Path
import numpy as np
from llm_assessor import (
    run_prompt_chain,
    extract_marks_from_graded_answer,
    construct_prompt_template
)
from langchain.prompts import PromptTemplate

load_dotenv(override=True)

In [ ]:
# Set working directory - Not required if using Jupyter outside of VScode
workdir = os.environ["workdir"]
os.chdir(workdir)
os.getcwd()

In [ ]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt="%Y-%m-%d %H:%M:%S",
    )


In [ ]:
# Load Directories containing Prompt Templates and Student Answers
prompt_dir = Path("prompt_templates")

data_dir = Path(f"{workdir}/validation_results/processed_data")

In [ ]:
# Load LLM
llm = ChatOpenAI(
    model_name="gpt-4o", 
    temperature=0.0, 
    max_tokens=500,
    )

# Load Data and Few Shot Examples

In [ ]:
# Weave examples into the recursive prompt
instruction = """
You are an exam grader. Your task is to grade exam questions according to a given mark scheme. 
You will be given a Question, Answer and a Mark Scheme to grade the Answer.
Follow these detailed instructions:
1. **Understand the Levels and Descriptors:**
- Review the mark scheme to understand the different levels and their descriptors. Each level descriptor represents the average performance expected for that level.

2. **Annotate the Answer:**
- Read through the student’s answer and annotate it based on the qualities outlined in the mark scheme. Highlight the key points that align with the descriptors.

3. **Determine a Level:**
- Start at the lowest level of the mark scheme and check if the answer meets the criteria for that level.
- If it does, move to the next level and repeat until you find the highest level the answer meets.
- Consider the overall quality of the answer, rather than focusing on minor errors.
- Use a best-fit approach if the answer includes elements from multiple levels, placing it in the level that best matches the predominant quality.
- Be strict with criteria for each level.

4. **Determine a Mark:**
- Once a level is assigned, decide on the specific mark within that level.
- Compare the student’s answer to exemplar materials provided during standardization, which include answers with assigned marks by the Lead Examiner.
- Use these comparisons to judge if the student's answer is of the same, better, or worse standard than the example, and assign a mark accordingly.

5. **Use Indicative Content as a Guide:**
- The indicative content in the mark scheme is a guide and not exhaustive. Credit valid points made by the student even if they are not listed in the indicative content.
- Students do not need to cover all points in the indicative content to achieve the highest level.

6. **Award No Marks for Irrelevant Answers:**
- If an answer contains nothing relevant to the question, award zero marks.
- If the answer is blank, award zero marks.

7. **Review as Necessary:**
- Re-read the student’s answer as needed to ensure the level and mark are appropriate and accurately reflect the answer’s quality.

By following these steps, you can accurately grade student answers according to the mark scheme, ensuring consistency and fairness in the evaluation process.
Report the Response/ Examiner Commentary only. Do not invent anything.
Below are examples to guide you. 
""".strip()

In [ ]:
judgement_few_shot_examples = [
    {
        "Question": "'The loss of territory was the main reason why Germans hated the Treaty of Versailles.'\nHow far do you agree with the statement.\nExplain your answer.",
        "Mark Scheme": "Level_4:\n  description: \n    - Complex explanation of stated factor and other factor(s) leading to a sustained judgement\n  marks_awarded:\n    maximum: 16\n    minimum: 13\n  criteria:\n    - Demonstrates a range of accurate and detailed knowledge and understanding that is relevant to the question\n    - Demonstrates a complex, sustained line of reasoning which has a sharply-focused coherence and logical structure that is fully substantiated, with well-judged relevance\n  progression: \n    - Extends Level 3\n    - Students may progress from a developed explanation of causation by complex explanation of the relationship between causes supported by detailed factual knowledge and understanding to form a sustained judgement\n  example_answers: \n    - The loss of territory was not the main reason because Germany expected some losses. What they hated the most about the Treaty of Versailles was that Germany was excluded from the negotiations and it was a \u2018Diktat\u2019. This meant they could not influence the severity of the resulting terms. This meant they could not prevent the inclusion of Clause 231 concerning war guilt which said they were responsible for starting the war.\nLevel_3:\n  description: \n    - Developed explanation of the stated factor and other factor(s)\n  marks_awarded:\n    maximum: 12\n    minimum: 9\n  criteria:\n    - Demonstrates a range of accurate knowledge and understanding that is relevant to the question\n    - Demonstrates a developed, sustained line of reasoning which has coherence and logical structure; it is well substantiated, and with sustained, explicit relevance\n  progression: \n    - Extends Level 2\n    - Answer may suggest that one reason has greater merit\n    - Students may progress from a simple explanation of causation with developed reasoning supported by factual knowledge and understanding\n  example_answers: \n    - The territorial terms of the Treaty of Versailles were the main reason why Germany hated it because they were humiliating and economically damaging. 10% of its land was given to neighbouring countries. A strip of land was given to Poland which effectively split Germany in two by creating a corridor that created access to the sea. As well as losing land outright, Germany also lost control of areas such as the coal rich Saarland which was put under a League of Nations mandate for 15 years and the Rhineland was demilitarised.\n    - The reparations payments were another reason why Germans hated the Treaty of Versailles because it would make it harder for Germany to recover. People were suffering from starvation after the war, and they were angry that the reparation bill was so huge. Germany had to pay money to the Allies for the damage caused during the war. It was estimated that it would take over 60 years to pay back the full bill.\nLevel_2:\n  description: \n    - Simple explanation of stated factor or other factor(s)\n  marks_awarded:\n    maximum: 8\n    minimum: 5\n  criteria:\n    - Demonstrates specific knowledge and understanding that is relevant to the question\n    - Demonstrates a simple, sustained line of reasoning which is coherent, structured, substantiated and explicitly relevant\n    - Answers arguing a preference for one judgement but with only basic explanation of another view will be marked at this level\n  progression: \n    - Students may progress from a basic explanation of causation by simple reasoning and supporting it with factual knowledge and understanding\n  example_answers: \n    - The War Guilt clause was hated by the German people because it forced them to accept the full blame for starting the war. Germany had no choice about signing the treaty; they were threatened with renewed fighting if they did not accept all the terms.\nLevel_1:\n  description: \n    - Basic explanation of one or more factors\n  marks_awarded:\n    maximum: 4\n    minimum: 1\n  criteria:\n    - Demonstrates basic knowledge and understanding that is relevant to the question\n    - Demonstrates a basic line of reasoning, which is coherent, structured with some substantiation; the relevance might be implicit\n    - Students recognise and provide a basic explanation of one or more factors\n  example_answers: \n    - The Treaty of Versailles took a lot of land from Germany.\n    - Germany was humiliated and forced to reduce the size of its army.\nLevel_0:\n  marks_awarded:\n    maximum: 0\n    minimum: 0\n  criteria:\n    - Students either submit no evidence or fail to address the question",
        "Answer": "I disagree with this statement as the loss of territory was a big reason why the Germans hated\nthe Treaty of Versailles, but it was not the worst sanction.\nArguably, the worst sanction for Germany was the reparations Germany had to pay which\nequated to 6.6 billion which would later cripple Germany\u2019s economy. This would leave\nGermany\u2019s people in a state of depression and make them a poor country.\nAnother bad sanction of the Treaty of Versailles would be the loss of Army. Germany\u2019s army\nwas decreased to only 100,000 men, no airforce and only 1 submarine.\nThis was crippling for Germany as it lost them open for attack and making them way less\npowerful. This sanction also meant that Britain had naval supremacy.\nFurthermore another sanction was that Germany had to take blame for starting the world\nwar. (Article 231) so if it wasn\u2019t bad enough that Germany lost all of its money and majority of\nits Army, they also had to take the blame for the war which left a lot of the German\u2019s furious.\nOverall, I disagree with the statement as the other sanctions were way more serious than just\nthe loss of land.",
        "Response": "The student moves beyond the \u2018basic explanation\u2019 of level 1, as they go beyond a stating of a factor to show some \u2018specific knowledge and understanding\u2019 of the factors, required for level 2. This is shown in the response in the second paragraph where the amount of the reparations is given and the impact of this is explained \u2018This would leave Germany\u2019s people in a state of depression and make them a poor country.\u2019 This is a simple explanation and does not reach the \u2018developed explanation\u2019 needed for level 3. \nFurther level 2 points are made, showing knowledge about the military restrictions in paragraph 3 \u2018100,000 men, no airforce and only 1 submarine. This was crippling for Germany as\u2026\u2019 and in the explanation of article 231. Three different factors are mentioned (financial, military and blame) but each of these is dealt with briefly so the answer remains in level 2. Multiple level 2 paragraphs do not add up to make level 3.\nLevel 3 also requires the student to explain \u2018the stated factor and other factor(s)\u2019. The stated factor \u2018loss of territory\u2019 is briefly mentioned and dismissed in the first sentence, but no knowledge or understanding is shown of this, so the answer is limited to level 2. Overall, the answer falls into level 2 and should be awarded 6 marks."
    },
    {
        "Question": "'The loss of territory was the main reason why Germans hated the Treaty of Versailles.'\nHow far do you agree with the statement.\nExplain your answer.",
        "Mark Scheme": "Level_4:\n  description: \n    - Complex explanation of stated factor and other factor(s) leading to a sustained judgement\n  marks_awarded:\n    maximum: 16\n    minimum: 13\n  criteria:\n    - Demonstrates a range of accurate and detailed knowledge and understanding that is relevant to the question\n    - Demonstrates a complex, sustained line of reasoning which has a sharply-focused coherence and logical structure that is fully substantiated, with well-judged relevance\n  progression: \n    - Extends Level 3\n    - Students may progress from a developed explanation of causation by complex explanation of the relationship between causes supported by detailed factual knowledge and understanding to form a sustained judgement\n  example_answers: \n    - The loss of territory was not the main reason because Germany expected some losses. What they hated the most about the Treaty of Versailles was that Germany was excluded from the negotiations and it was a \u2018Diktat\u2019. This meant they could not influence the severity of the resulting terms. This meant they could not prevent the inclusion of Clause 231 concerning war guilt which said they were responsible for starting the war.\nLevel_3:\n  description: \n    - Developed explanation of the stated factor and other factor(s)\n  marks_awarded:\n    maximum: 12\n    minimum: 9\n  criteria:\n    - Demonstrates a range of accurate knowledge and understanding that is relevant to the question\n    - Demonstrates a developed, sustained line of reasoning which has coherence and logical structure; it is well substantiated, and with sustained, explicit relevance\n  progression: \n    - Extends Level 2\n    - Answer may suggest that one reason has greater merit\n    - Students may progress from a simple explanation of causation with developed reasoning supported by factual knowledge and understanding\n  example_answers: \n    - The territorial terms of the Treaty of Versailles were the main reason why Germany hated it because they were humiliating and economically damaging. 10% of its land was given to neighbouring countries. A strip of land was given to Poland which effectively split Germany in two by creating a corridor that created access to the sea. As well as losing land outright, Germany also lost control of areas such as the coal rich Saarland which was put under a League of Nations mandate for 15 years and the Rhineland was demilitarised.\n    - The reparations payments were another reason why Germans hated the Treaty of Versailles because it would make it harder for Germany to recover. People were suffering from starvation after the war, and they were angry that the reparation bill was so huge. Germany had to pay money to the Allies for the damage caused during the war. It was estimated that it would take over 60 years to pay back the full bill.\nLevel_2:\n  description: \n    - Simple explanation of stated factor or other factor(s)\n  marks_awarded:\n    maximum: 8\n    minimum: 5\n  criteria:\n    - Demonstrates specific knowledge and understanding that is relevant to the question\n    - Demonstrates a simple, sustained line of reasoning which is coherent, structured, substantiated and explicitly relevant\n    - Answers arguing a preference for one judgement but with only basic explanation of another view will be marked at this level\n  progression: \n    - Students may progress from a basic explanation of causation by simple reasoning and supporting it with factual knowledge and understanding\n  example_answers: \n    - The War Guilt clause was hated by the German people because it forced them to accept the full blame for starting the war. Germany had no choice about signing the treaty; they were threatened with renewed fighting if they did not accept all the terms.\nLevel_1:\n  description: \n    - Basic explanation of one or more factors\n  marks_awarded:\n    maximum: 4\n    minimum: 1\n  criteria:\n    - Demonstrates basic knowledge and understanding that is relevant to the question\n    - Demonstrates a basic line of reasoning, which is coherent, structured with some substantiation; the relevance might be implicit\n    - Students recognise and provide a basic explanation of one or more factors\n  example_answers: \n    - The Treaty of Versailles took a lot of land from Germany.\n    - Germany was humiliated and forced to reduce the size of its army.\nLevel_0:\n  marks_awarded:\n    maximum: 0\n    minimum: 0\n  criteria:\n    - Students either submit no evidence or fail to address the question",
        "Answer": "I think the main reason why the Germans hated the treaty of Versailles was their loss of land.\nI agree that the Germans hated the treaty of Versailles due to the loss of territory as the\nGermans lost 10% of their land. They lost the Saar for 15 years as it was given control to\nFrance and the League of Nations. The Saar was a large industrial part of Germany that would\nof helped them pay their \u00a36,600 million reparations. They only got it back 15 years later when\nthe plebiscite voted 90% for Germany. They also lost their colonies in Africa which were given\nto Britain and provided them with lots of money and resources.\nHowever their loss of military cover be the main reason why the Germans hated the treaty of\nVersailles. Their army was reduced to 100,000 men and their navy reduced to 15,000. They\nwere not allowed an air force or submarines. The Rhineland was a strip of land on the border\nof France and Germany and the treaty of Versailles made them demilitarise the Rhineland.\nThis left Germany feeling vulnerable to attack as it was well known that Georges Clemenceau\nwanted revenge on Germany. This also led to Germans turning to Hitler as he promised a\nstrong army and rearmaments. He increased army to 1,000,000 and built his Luftwaffe This\naggression comes from their previous loss of their military.\nOverall I believe that the loss of territory was the main reason for the Germans hatred of the\ntreaty of Versailles as they lost the large industrial Saar as well as their colonies and they lost\n6,000,000 German speaking people due to the 10% of their land lost.",
        "Response": "The first paragraph addresses the \u2018stated factor\u2019 and is \u2018supported by factual knowledge and understanding\u2019, giving several examples of pieces of territory that were lost. However, the \u2018developed reasoning\u2019 required for level 3 is missing in this paragraph. To reach level 3, this paragraph would need to include an explanation of why those losses would make Germans hate the Treaty of Versailles. This is implied \u2018provided them with lots of money and resources\u2019 but not clearly explained. \nIn the second paragraph, the \u2018factual knowledge\u2019 and \u2018developed reasoning\u2019 is evident, as the student provides factual details from the military clauses of the Treaty of Versailles and explains the impact of these on the Germans. \u2018This left Germany feeling vulnerable\u2019. Developed explanation comes as they explain the longer term impact of this \u2018This also led to Germans turning to Hitler\u2019, \u2018This aggression comes from their previous loss of their military\u2019. This is a level 3 paragraph.\nThe conclusion makes a judgement, but is repetitive of earlier points. The quality of the conclusion is only relevant if an answer is secure in level 3, as a reasoned judgement is required for level 4 only. With one level 2 paragraph and one level 3 paragraph, this answer falls into level 3 and should be awarded 7 marks."
    },
    {
        "Question": "'The loss of territory was the main reason why Germans hated the Treaty of Versailles.'\nHow far do you agree with the statement.\nExplain your answer.",
        "Mark Scheme": "Level_4:\n  description: \n    - Complex explanation of stated factor and other factor(s) leading to a sustained judgement\n  marks_awarded:\n    maximum: 16\n    minimum: 13\n  criteria:\n    - Demonstrates a range of accurate and detailed knowledge and understanding that is relevant to the question\n    - Demonstrates a complex, sustained line of reasoning which has a sharply-focused coherence and logical structure that is fully substantiated, with well-judged relevance\n  progression: \n    - Extends Level 3\n    - Students may progress from a developed explanation of causation by complex explanation of the relationship between causes supported by detailed factual knowledge and understanding to form a sustained judgement\n  example_answers: \n    - The loss of territory was not the main reason because Germany expected some losses. What they hated the most about the Treaty of Versailles was that Germany was excluded from the negotiations and it was a \u2018Diktat\u2019. This meant they could not influence the severity of the resulting terms. This meant they could not prevent the inclusion of Clause 231 concerning war guilt which said they were responsible for starting the war.\nLevel_3:\n  description: \n    - Developed explanation of the stated factor and other factor(s)\n  marks_awarded:\n    maximum: 12\n    minimum: 9\n  criteria:\n    - Demonstrates a range of accurate knowledge and understanding that is relevant to the question\n    - Demonstrates a developed, sustained line of reasoning which has coherence and logical structure; it is well substantiated, and with sustained, explicit relevance\n  progression: \n    - Extends Level 2\n    - Answer may suggest that one reason has greater merit\n    - Students may progress from a simple explanation of causation with developed reasoning supported by factual knowledge and understanding\n  example_answers: \n    - The territorial terms of the Treaty of Versailles were the main reason why Germany hated it because they were humiliating and economically damaging. 10% of its land was given to neighbouring countries. A strip of land was given to Poland which effectively split Germany in two by creating a corridor that created access to the sea. As well as losing land outright, Germany also lost control of areas such as the coal rich Saarland which was put under a League of Nations mandate for 15 years and the Rhineland was demilitarised.\n    - The reparations payments were another reason why Germans hated the Treaty of Versailles because it would make it harder for Germany to recover. People were suffering from starvation after the war, and they were angry that the reparation bill was so huge. Germany had to pay money to the Allies for the damage caused during the war. It was estimated that it would take over 60 years to pay back the full bill.\nLevel_2:\n  description: \n    - Simple explanation of stated factor or other factor(s)\n  marks_awarded:\n    maximum: 8\n    minimum: 5\n  criteria:\n    - Demonstrates specific knowledge and understanding that is relevant to the question\n    - Demonstrates a simple, sustained line of reasoning which is coherent, structured, substantiated and explicitly relevant\n    - Answers arguing a preference for one judgement but with only basic explanation of another view will be marked at this level\n  progression: \n    - Students may progress from a basic explanation of causation by simple reasoning and supporting it with factual knowledge and understanding\n  example_answers: \n    - The War Guilt clause was hated by the German people because it forced them to accept the full blame for starting the war. Germany had no choice about signing the treaty; they were threatened with renewed fighting if they did not accept all the terms.\nLevel_1:\n  description: \n    - Basic explanation of one or more factors\n  marks_awarded:\n    maximum: 4\n    minimum: 1\n  criteria:\n    - Demonstrates basic knowledge and understanding that is relevant to the question\n    - Demonstrates a basic line of reasoning, which is coherent, structured with some substantiation; the relevance might be implicit\n    - Students recognise and provide a basic explanation of one or more factors\n  example_answers: \n    - The Treaty of Versailles took a lot of land from Germany.\n    - Germany was humiliated and forced to reduce the size of its army.\nLevel_0:\n  marks_awarded:\n    maximum: 0\n    minimum: 0\n  criteria:\n    - Students either submit no evidence or fail to address the question",
        "Answer": "On one hand, I can agree with this statement because in total 13% of Germany\u2019s land was\ngiven away and was taken. This meant that many German people and German speaking\npeople found themselves out of Germany. All of Germany\u2019s colonies in Africa were taken away\nas mandates but in reality, they were given to Britain and France. Additionally Alsace Lorraine\nwas returned to France and the Rhineland was demilitarised meaning even parts of Germany\nitself wasn\u2019t properly controlled by Germany. The Saar was also taken which contained lost of\nindustrial business. This meant that the once great power of Germany felt stripped of its\npower and many German people were forced to be a part of another new country.\nHowever, War guilt could be seen as the main reason why the Germans hated the treaty so\nmuch because firstly, article 231, the war guilt clause pointed all the blame on Germany and\nthe German people. This made the German population feel humiliated and they thought it\nwas unfair because the Kaiser and the German government started the war, not the German\npeople and how that the Kaiser had been humiliated and fled to Holland and they had a new\nWeimar republic, they believed that the blame shouldn\u2019t solely be on Germany. The people of\nGermany believe that the \u2018November criminals\u2019 signed the \u2018diktat\u2019 and it was their fault\nhowever, they did not understand that the government did not have a choice and have to sign\nthe armistice or war would have continued.\nThe Germans also hated the reparations and restrictions from the military. Firstly the\nGermans were made to pay \u00a36.6 billion in reparations to pay for the damage of the war.\nGermany was already struggling at the time (and people were homeless and dying of\nstarvation of starvation. These reparations mostly effected the people who were not to blame\nfor the war. Also the military, which used to be a proud part of Germany was restricted to\n100,000 men, 6 battleships, 15,000 sailors in the navy, no airforce and no submarines. This\ngave Britain naval supremacy which was humiliating for the Germans and only gave them\nenough army to defend themselves.\nWhat the Germans did not understand is that the treaty could have been much harsher, but\nthe humiliation of the war guilt was probably the main factor because it hit the German\npeople hard and made them outcasts to the rest of Europe and the fact that they were not\nallowed to join the League of Nations probably made it worse for them. The Treaty of\nVersailles should have been about the Government and the military leaders but instead it hit\nthe ordinary people the most.",
        "Response": "The student clearly shows developed explanation of the stated factor in the first paragraph and then two other factors. Each paragraph contains a range of \u2018factual knowledge\u2019 and \u2018developed reasoning\u2019 as the student explains why each factor would make the Germans hate the Treaty of Versailles. This puts the answer securely into level 3.\nLevel 4 requires \u2018complex explanation\u2019, a \u2018logical structure\u2019 and \u2018sustained line of reasoning\u2019 to form a \u2018sustained judgement\u2019. The conclusion is not a repetition of earlier points, it attempts to draw them together and consider the overall impact on the Germans and the injustice of the people being punished when the leaders were to blame. The theme of humiliation is sustained throughout \u2018Germany felt stripped of its power\u2019, \u2018This made the German population feel humiliated\u2019 \u2018These reparations mostly affected the people who were not to blame for the war\u2019, \u2018which was humiliating for the Germans\u2019. It is this sustained judgement and the recognition of the difference between the leaders and the people which puts this answer into level 4. The structure is clear and coherent but lacks some complexity and comparisons between different factors throughout, so wouldn\u2019t reach the top of the level. Overall, this falls into level 4 and should be awarded 14 marks."
    },
    {
        "Question": "The U2 Crisis was the main reason why tension developed between East and West during the 1960s.\u2019 How far do you agree with this statement? Explain your answer.",
        "Mark Scheme": "Level 4:\n  description: \n    - Complex explanation of stated factor and other factor(s) leading to a sustained judgement\n    - Answer demonstrates a range of accurate and detailed knowledge and understanding that is relevant to the question\n    - Answer demonstrates a complex, sustained line of reasoning which has a sharply-focused coherence and logical structure that is fully substantiated, with well-judged relevance.\n  marks_awarded:\n    maximum: 16\n    minimum: 13\n  progression: \n    - Extends Level 3.\n    - Students may progress from a developed explanation of causation by complex explanation of the relationship between causes supported by detailed factual knowledge and understanding to form a sustained judgement.\n  example_answers: \n    - The U2 Crisis was not the main reason why tension developed during the 1960s because more serious conflicts followed such as the loss of life incurred during the Prague Spring and the international fear of nuclear war during the Cuban Missile Crisis. However, coming as it did at the very start of the decade, it could be argued that it set the tone for East\u2013West relations by making it clear that \u2018peaceful co\u2013existence\u2019 was unlikely.\nLevel 3:\n  description: \n    - Developed explanation of the stated factor and other factor(s)\n    - Answer demonstrates a range of accurate knowledge and understanding that is relevant to the question\n    - Answer demonstrates a developed, sustained line of reasoning which has coherence and logical structure; it is well substantiated, and with sustained, explicit relevance.\n  marks_awarded:\n    maximum: 12\n    minimum: 9\n  progression: \n    - Extends Level 2.\n    - Answer may suggest that one reason has greater merit.\n    - Students may progress from a simple explanation of causation with developed reasoning supported by factual knowledge and understanding.\n  example_answers: \n    - The U2 Crisis was the main reason why tension developed between the East and West because once the spy plane had been captured by the Soviets, the American government lied about the purpose of the flight. They claimed it was simply conducting weather research. Khrushchev revealed the deception at the opening of the Paris Peace summit and when Eisenhower refused to apologise, the meeting was abandoned. This affected the Cold War because it was a missed opportunity to have face-to-face discussions and reduce tension.\n    - The Prague Spring was also a reason why tension developed during the 1960s because Soviet forces invaded Czechoslovakia and installed their own loyal government. Dubcek had introduced political reforms into Czechoslovakia by relaxing state control of the economy and the press. The USSR and the other Warsaw Pact leaders felt any reforms could damage the communist system as a whole. East\u2013West tension increased when Western countries criticised the violent response from Moscow and called for the UN to condemn the USSR.\nLevel 2:\n  description: \n    - Simple explanation of stated factor or other factor(s)\n    - Answer demonstrates specific knowledge and understanding that is relevant to the question\n    - Answer demonstrates a simple, sustained line of reasoning which is coherent, structured, substantiated and explicitly relevant.\n  marks_awarded:\n    maximum: 8\n    minimum: 5\n  progression: \n    - Answers arguing a preference for one judgement but with only basic explanation of another view will be marked at this level.\n    - Students may progress from a basic explanation of causation by simple reasoning and supporting it with factual knowledge and understanding.\n  example_answers: \n    - Tension developed during the 1960s because of the Cuban Missile Crisis. The USSR had nuclear missiles on the island of Cuba which put America in danger of attack. Kennedy used a naval blockade to stop any more missiles arriving on Cuba and demanded that the existing missiles were removed.\nLevel 1:\n  description: \n    - Basic explanation of one or more factors\n    - Answer demonstrates basic knowledge and understanding that is relevant to the question\n    - Answer demonstrates a basic line of reasoning, which is coherent, structured with some substantiation; the relevance might be implicit.\n  marks_awarded:\n    maximum: 4\n    minimum: 1\n  progression: \n    - Students recognise and provide a basic explanation of one or more factors.\n  example_answers: \n    - Tension developed between East and West during the 1960s because an American U2 plane was caught spying on the USSR.\n    - There was an increase in tension when USSR built the Berlin Wall. No one knew how the USA would react. Fighting may have broken out.\nLevel 0:\n  description: \n    - Students either submit no evidence or fail to address the question\n  marks_awarded:\n    maximum: 0\n    minimum: 0",
        "Answer": "The U2 crisis was a major source of tension between USSR and the USA during the 1960\u2019s. The U2 spy plane made USSR think that the USA was spying on them, which they were, and cause a major increase in the lack of trust between USA and USSR.\nAt the time of the U2 spy plane the space race was also going on at the same time. The space race took up major amounts of money for USA and USSR. The risk of USA finding out USSR secrets with the spy plane would mean major loss for the USSR.\nKhrushchev was very worried about the plane so he shot it down. The pilot refused to say anything so he was killed.\nThe Space race was a major source of tension as it was a way to prove which leadership is strongest USA or USSR.\nDuring the 1960s the Cuban Missile Crisis was also happening which was major tension as Cuba is just 90 miles off the coast of Florida and USSR control so close was very dangerous for the USA. During the CMC the world was at the closest point it has ever been to nuclear war.\nI agree with this statement because the spy plane was a direct threat to the USSR and it caused huge tension between USA and USSR as it was clear evidence that USA had been spying on them and that the USA could have information about the amount of weapons they have and also the development of the space race in the USSR.",
        "Response": "The student addresses the \u2018given factor\u2019 (the U2 crisis), providing a brief summary of events and explaining the impact of this \u2018a major increase in the lack of trust between USA and USSR\u2019. They then go on to explain different alternative factors, the space race and the Cuban Missile Crisis, both accurate reasons why tension developed during the 1960s. The structure of the answer could open it up to level 3, but throughout the student displays \u2018simple explanation\u2019 rather than \u2018developed explanation\u2019. There is some \u2018specific knowledge and understanding\u2019 of each of the events, but this doesn\u2019t develop into the \u2018range of accurate knowledge\u2019 required for level 3.\nTo get to level 3, the answer would need to be better structured, showing \u2018coherence and logical structure\u2019, as it is, the student moves between factors in a series of short paragraphs and flips between the U2 crisis and space race.  \nThere is a conclusion at the end of the answer, which makes a judgement but the conclusion is only really relevant to reach level 4 once level 3 has been secured, and as explained above, the answer falls short of level 3 in the depth of its explanations.\nOverall, the answer falls into level 2 and should be awarded 7 marks."
    },
    {
        "Question": "The U2 Crisis was the main reason why tension developed between East and West during the 1960s.\u2019 How far do you agree with this statement? Explain your answer.",
        "Mark Scheme": "Level 4:\n  description: \n    - Complex explanation of stated factor and other factor(s) leading to a sustained judgement\n    - Answer demonstrates a range of accurate and detailed knowledge and understanding that is relevant to the question\n    - Answer demonstrates a complex, sustained line of reasoning which has a sharply-focused coherence and logical structure that is fully substantiated, with well-judged relevance.\n  marks_awarded:\n    maximum: 16\n    minimum: 13\n  progression: \n    - Extends Level 3.\n    - Students may progress from a developed explanation of causation by complex explanation of the relationship between causes supported by detailed factual knowledge and understanding to form a sustained judgement.\n  example_answers: \n    - The U2 Crisis was not the main reason why tension developed during the 1960s because more serious conflicts followed such as the loss of life incurred during the Prague Spring and the international fear of nuclear war during the Cuban Missile Crisis. However, coming as it did at the very start of the decade, it could be argued that it set the tone for East\u2013West relations by making it clear that \u2018peaceful co\u2013existence\u2019 was unlikely.\nLevel 3:\n  description: \n    - Developed explanation of the stated factor and other factor(s)\n    - Answer demonstrates a range of accurate knowledge and understanding that is relevant to the question\n    - Answer demonstrates a developed, sustained line of reasoning which has coherence and logical structure; it is well substantiated, and with sustained, explicit relevance.\n  marks_awarded:\n    maximum: 12\n    minimum: 9\n  progression: \n    - Extends Level 2.\n    - Answer may suggest that one reason has greater merit.\n    - Students may progress from a simple explanation of causation with developed reasoning supported by factual knowledge and understanding.\n  example_answers: \n    - The U2 Crisis was the main reason why tension developed between the East and West because once the spy plane had been captured by the Soviets, the American government lied about the purpose of the flight. They claimed it was simply conducting weather research. Khrushchev revealed the deception at the opening of the Paris Peace summit and when Eisenhower refused to apologise, the meeting was abandoned. This affected the Cold War because it was a missed opportunity to have face-to-face discussions and reduce tension.\n    - The Prague Spring was also a reason why tension developed during the 1960s because Soviet forces invaded Czechoslovakia and installed their own loyal government. Dubcek had introduced political reforms into Czechoslovakia by relaxing state control of the economy and the press. The USSR and the other Warsaw Pact leaders felt any reforms could damage the communist system as a whole. East\u2013West tension increased when Western countries criticised the violent response from Moscow and called for the UN to condemn the USSR.\nLevel 2:\n  description: \n    - Simple explanation of stated factor or other factor(s)\n    - Answer demonstrates specific knowledge and understanding that is relevant to the question\n    - Answer demonstrates a simple, sustained line of reasoning which is coherent, structured, substantiated and explicitly relevant.\n  marks_awarded:\n    maximum: 8\n    minimum: 5\n  progression: \n    - Answers arguing a preference for one judgement but with only basic explanation of another view will be marked at this level.\n    - Students may progress from a basic explanation of causation by simple reasoning and supporting it with factual knowledge and understanding.\n  example_answers: \n    - Tension developed during the 1960s because of the Cuban Missile Crisis. The USSR had nuclear missiles on the island of Cuba which put America in danger of attack. Kennedy used a naval blockade to stop any more missiles arriving on Cuba and demanded that the existing missiles were removed.\nLevel 1:\n  description: \n    - Basic explanation of one or more factors\n    - Answer demonstrates basic knowledge and understanding that is relevant to the question\n    - Answer demonstrates a basic line of reasoning, which is coherent, structured with some substantiation; the relevance might be implicit.\n  marks_awarded:\n    maximum: 4\n    minimum: 1\n  progression: \n    - Students recognise and provide a basic explanation of one or more factors.\n  example_answers: \n    - Tension developed between East and West during the 1960s because an American U2 plane was caught spying on the USSR.\n    - There was an increase in tension when USSR built the Berlin Wall. No one knew how the USA would react. Fighting may have broken out.\nLevel 0:\n  description: \n    - Students either submit no evidence or fail to address the question\n  marks_awarded:\n    maximum: 0\n    minimum: 0",
        "Answer": "During the 1960\u2019s although the U2 Crisis did cause tension to develop between the East and the West. I think other factors like the Cuban Missile Crisis may have been the main reason for the development of tension between the two superpowers.\nIn 1960, an American pilot, Gary Powers \u2013 was flying the U2 spy plane over the USSR. This plane was shot down by the Soviets when it was flying closer to the ground and both Powers\u2019 and the plan was captured. This all took place while the Paris peace summit was also ongoing. When America was questioned by the Soviet\u2019s \u2013 for multiple days. Eisenhower lied and used the excuse that it was a weather plane and denied any wrongdoing or espionage. After a few days the Soviets assessed the wreckage of the U2 plane and discovered technology like cameras that could read a newspaper from 20,000 feet in the air \u2013 monitoring equipment and even a cyanide pill. All of this equipment indicated that it was in fact a spy plane and Eisenhower had lied to Khrushchev\u2019s face. At the Paris peace summit talks Eisenhower then refused to apologise for the U2 spy plane which caused Khrushchev to walk out of the talks and ended the peace talks and ended any chance of the \u2018thaw\u2019. This increased tension as it was clear that espionage and spying had been taking place in the Soviet skies with all of the improved technology and Eisenhower had lied to Khrushchev and refused to show any remorse for their decisions showing that America did not feel in the wrong about the spying. This then infuriated the Soviet\u2019s and increased the distrust between the two superpowers and led to a spike in tension.\nOn the other hand, events in 1962 led to a massive increase in tension during the event. In the early 1960\u2019s Fidel Castro came to power in Cuba as a big communist believer. Geographically Cuba is quite near to the USA and was a big difference to the spread of communism in the East. Once the Soviet\u2019s started to get involved with Castro and the leading of Cuba, America were seeing this as a failure of the Truman Doctrine and containment of Communism. During the Bay of Pigs \u2013 John. F. Kennedy decided that they were going to threaten to stop exporting their sugar supply that was mainly from Cuba and also started putting blockades up against any Soviet ships trying to enter. This event between John. F. Kennedy and Khrushchev was the closest the world had ever come to Nuclear war which would not just affect the two superpowers of USA and USSR but in fact the whole of the East and West. Although it never reached the extremity of Nuclear war because of the reminder and realisation of Mutually Assured Destruction, it was the highest that tension and the threat between the East and the West.\nIn conclusion, I believe that the U2 crisis was not the main reason for tension but actually the Cuban Missile Crisis as although there was a Test Ban treaty in 1963 during the Cuban Missile Crisis tension was at its highest than ever before and was not just affecting the two superpowers mainly like the U2 crisis but the whole of the East and West with threats of complete Nuclear War.",
        "Response": "The student starts with a brief introduction, which, although not required, helps to provide the sustained judgement which is aimed for in level 4.\nParagraph 1 explains the given factor in a developed way. There is a range of \u2018accurate factual knowledge\u2019 shown about the events and the student then explains how that led to an increase in tension \u2018This increased tension as it was clear that espionage and spying had been taking place\u2019; \u2018this then infuriated the soviets and increased the distrust\u2026. Led to a spike in tension\u2019. This is a developed line of reasoning, with several aspects considered and the consequences taken to their conclusion.\nParagraph 2 explains an alternative factor \u2013 the Cuban Missile Crisis. Again, there is a range of \u2018factual knowledge and understanding\u2019 of events, however the explanation and focus on the issue of increased tension is weaker here, it is hinted at in the final sentence \u2018it was the highest that tension and the threat between the East and the West\u2019, however this is \u2018simple explanation\u2019. Level 2 allows for \u2018coherent, structured, substantiated and explicitly relevant\u2019 which this is. The second paragraph lacks the \u2018developed sustained line of reasoning\u2019 shown in paragraph 1. However, the presence of one level 3 paragraph puts the answer into level 3, even if the second paragraph is weaker.\nThe conclusion repeats the judgement made in the introduction, but as there isn\u2019t two secure level 3 paragraphs, the quality of the conclusion isn\u2019t relevant to the level attained and the points made are repeated from earlier paragraphs, rather than adding anything to the judgement.\nOverall, the answer falls into level 3 and should be awarded 8 marks."
    },
    {
        "Question": "The U2 Crisis was the main reason why tension developed between East and West during the 1960s.\u2019 How far do you agree with this statement? Explain your answer.",
        "Mark Scheme": "Level 4:\n  description: \n    - Complex explanation of stated factor and other factor(s) leading to a sustained judgement\n    - Answer demonstrates a range of accurate and detailed knowledge and understanding that is relevant to the question\n    - Answer demonstrates a complex, sustained line of reasoning which has a sharply-focused coherence and logical structure that is fully substantiated, with well-judged relevance.\n  marks_awarded:\n    maximum: 16\n    minimum: 13\n  progression: \n    - Extends Level 3.\n    - Students may progress from a developed explanation of causation by complex explanation of the relationship between causes supported by detailed factual knowledge and understanding to form a sustained judgement.\n  example_answers: \n    - The U2 Crisis was not the main reason why tension developed during the 1960s because more serious conflicts followed such as the loss of life incurred during the Prague Spring and the international fear of nuclear war during the Cuban Missile Crisis. However, coming as it did at the very start of the decade, it could be argued that it set the tone for East\u2013West relations by making it clear that \u2018peaceful co\u2013existence\u2019 was unlikely.\nLevel 3:\n  description: \n    - Developed explanation of the stated factor and other factor(s)\n    - Answer demonstrates a range of accurate knowledge and understanding that is relevant to the question\n    - Answer demonstrates a developed, sustained line of reasoning which has coherence and logical structure; it is well substantiated, and with sustained, explicit relevance.\n  marks_awarded:\n    maximum: 12\n    minimum: 9\n  progression: \n    - Extends Level 2.\n    - Answer may suggest that one reason has greater merit.\n    - Students may progress from a simple explanation of causation with developed reasoning supported by factual knowledge and understanding.\n  example_answers: \n    - The U2 Crisis was the main reason why tension developed between the East and West because once the spy plane had been captured by the Soviets, the American government lied about the purpose of the flight. They claimed it was simply conducting weather research. Khrushchev revealed the deception at the opening of the Paris Peace summit and when Eisenhower refused to apologise, the meeting was abandoned. This affected the Cold War because it was a missed opportunity to have face-to-face discussions and reduce tension.\n    - The Prague Spring was also a reason why tension developed during the 1960s because Soviet forces invaded Czechoslovakia and installed their own loyal government. Dubcek had introduced political reforms into Czechoslovakia by relaxing state control of the economy and the press. The USSR and the other Warsaw Pact leaders felt any reforms could damage the communist system as a whole. East\u2013West tension increased when Western countries criticised the violent response from Moscow and called for the UN to condemn the USSR.\nLevel 2:\n  description: \n    - Simple explanation of stated factor or other factor(s)\n    - Answer demonstrates specific knowledge and understanding that is relevant to the question\n    - Answer demonstrates a simple, sustained line of reasoning which is coherent, structured, substantiated and explicitly relevant.\n  marks_awarded:\n    maximum: 8\n    minimum: 5\n  progression: \n    - Answers arguing a preference for one judgement but with only basic explanation of another view will be marked at this level.\n    - Students may progress from a basic explanation of causation by simple reasoning and supporting it with factual knowledge and understanding.\n  example_answers: \n    - Tension developed during the 1960s because of the Cuban Missile Crisis. The USSR had nuclear missiles on the island of Cuba which put America in danger of attack. Kennedy used a naval blockade to stop any more missiles arriving on Cuba and demanded that the existing missiles were removed.\nLevel 1:\n  description: \n    - Basic explanation of one or more factors\n    - Answer demonstrates basic knowledge and understanding that is relevant to the question\n    - Answer demonstrates a basic line of reasoning, which is coherent, structured with some substantiation; the relevance might be implicit.\n  marks_awarded:\n    maximum: 4\n    minimum: 1\n  progression: \n    - Students recognise and provide a basic explanation of one or more factors.\n  example_answers: \n    - Tension developed between East and West during the 1960s because an American U2 plane was caught spying on the USSR.\n    - There was an increase in tension when USSR built the Berlin Wall. No one knew how the USA would react. Fighting may have broken out.\nLevel 0:\n  description: \n    - Students either submit no evidence or fail to address the question\n  marks_awarded:\n    maximum: 0\n    minimum: 0",
        "Answer": "One of the reasons for tensions developing between East and West during the 1960s was the U2 Crisis. This was because the U2 Crisis had the potential to damage relations irreparably. The U2 Crisis was where a US spy plane was spotted and shot down in Soviet territory. Eisenhower at first claimed that the plane was a weather plane, this was proven to be false, however, as there were cameras on the plane and the pilot, Gary Powers, was carrying a cynide pill. When America admitted to it being a spy plane no apology was given, nor was it promised to not happen again. This increased tensions as it led to Krushchev walking out of the Paris Peace Summit shortly after. While this crisis did develop tensions it did not increase them to the point of potential war and therefore it was not the main reason for tensions developing between East and West during the 1960s.\nA final reason for tensions developing between East and West during the 1960\u2019s was the Cuban Missile Crisis. This was because the Cuban Missile Crisis has the potential to cause a nuclear war. The Cuban Missile Crisis was where the USSR was found to be keeping weapons, like ICBMs in Cubs. This worried the USA as Cuba was extremely close to it compared to where the Soviet Union was to the USA. This meant that the Soviet Union could easily and quickly destroy USA cities due to the close range. The USA responded to this by causing what was essentially a naval quarantine on ships heading to Cuba where they would check the ships for military weapons before letting them through. At this point the USA had reached DEFCON 2, the highest risk of nuclear war it has ever seen. Thankfully, both sides decided to back down with the USSR removing its weapons from Cuba and the USA removing its weapons from Turkey. This was the main reason for tensions as it risked nuclear war.\nOverall, the Cuban Missile Crisis was the main reason for tensions developing between East and West during the 1960s. While both the Berlin Wall and the O2 crisis did increase tensions, they did not raise them to the point that the Cuban Missile Crisis did, nor did they risk the total destruction that it did either therefore it was Cuba that was the main reason for tensions developing in the 1960\u2019s. The Cuban Missile Crisis highlighted to potential that the idea of Mutually Assured Destruction (M.A.D) could become a reality.",
        "Response": "The student addresses the given factor in the first paragraph and shows a \u2018range of accurate factual knowledge and understanding\u2019 and a clear link to the question, with \u2018this increased tensions as it led to\u2026\u2019. This paragraph then starts to develop a judgement by evaluating the severity of the risk and the tensions at this point. \u2018it did not increase them to the point of potential war and therefore it was not the main reason\u2026.\u2019. This sentence is a good indicator of a potential level 4 answer as the student is starting to build a sustained judgement and setting out criteria for a complex, sustained line of reasoning.\nIn paragraph 2 (it seems like a middle paragraph about the Berlin Wall may have been removed here to keep the answer to a reasonable length for the exercise), the student explains an alternative factor \u2013 the Cuban Missile Crisis. Again, there is factual knowledge demonstrated at a good level of detail. The student explains the level of tension \u2018the USA had reached DEFCON 2\u2019 and makes a judgement at the end of the paragraph \u2018this was the main reason for tensions as it risked nuclear war\u2019. This is a second level 3 paragraph.\nThe conclusion secures level 4 by comparing the relative severity of the different events \u2018while both the Berlin Wall and the O2 crisis did raise tensions, they did not\u2026 risk the total destruction\u2019 of the Cuban Missile Crisis. This line of reasoning has been sustained throughout the answer, and is then brought together in the conclusion.\nOverall, this answer falls into level 4 and should be awarded 15 marks."
    },
    {
        "Question": "\u2018The Gulf of Tonkin incident was the main reason why the conflict in Vietnam escalated in the 1960s.\u2019 How far do you agree with this statement? Explain your answer.",
        "Mark Scheme": "Level_4:\n  description: \n    - Complex explanation of stated factor and other factor(s) leading to a sustained judgement\n    - Answer demonstrates a range of accurate and detailed knowledge and understanding that is relevant to the question.\n    - Answer demonstrates a complex, sustained line of reasoning which has a sharply focused coherence and logical structure that is fully substantiated, with well-judged relevance.\n  marks_awarded:\n    maximum: 16\n    minimum: 13\n  progression: \n    - Extends Level 3.\n    - Students may progress from a developed explanation of causation by complex explanation of the relationship between causes supported by detailed factual knowledge and understanding to form a sustained judgement.\n  example_answers: \n    - The Gulf of Tonkin incident was just one of the reasons for escalation in the 1960s. The wider context was that as the war continued, the nature of the conflict made decisive victory impossible. The size and strength of US military power did not bring a great advantage in jungle warfare against the guerrilla tactics of the Vietcong, nor could it destroy the Ho Chi Minh trail.\nLevel_3:\n  description: \n    - Developed explanation of the stated factor and other factor(s)\n    - Answer demonstrates a range of accurate knowledge and understanding that is relevant to the question\n    - Answer demonstrates a developed, sustained line of reasoning which has coherence and logical structure; it is well substantiated, and with sustained, explicit relevance.\n  marks_awarded:\n    maximum: 12\n    minimum: 9\n  progression: \n    - Extends Level 2.\n    - Answer may suggest that one reason has greater merit.\n    - Students may progress from a simple explanation of causation with developed reasoning supported by factual knowledge and understanding.\n  example_answers: \n    - The Gulf of Tonkin incident involved a torpedo attack on USS Maddox and the report of another imminent attack. Although no further attacks occurred, the suspicion was sufficient for the US government to pass a resolution which gave the President the power to take further military against North Vietnam. This escalated the conflict because President Johnson used this power to launch bombing campaigns against North Vietnam and sent thousands of American soldiers to South Vietnam.\n    - The main reason the conflict continued in the 1960s was because the Vietcong successfully fought a guerrilla war. The Vietcong hung on the belts of the Americans. The Americans struggled to identify the enemy because they didn\u2019t wear uniforms and hid in the jungle and launched ambush attacks. The Americans could not be seen to be losing so they kept sending more forces and used new methods of attack such as Agent Orange to clear the forest.\nLevel_2:\n  description: \n    - Simple explanation of stated factor or other factor(s)\n    - Answer demonstrates specific knowledge and understanding that is relevant to the question\n    - Answer demonstrates a simple, sustained line of reasoning which is coherent, structured, substantiated and explicitly relevant.\n    - Answers arguing a preference for one judgement but with only basic explanation of another view will be marked at this level.\n  marks_awarded:\n    maximum: 8\n    minimum: 5\n  progression: \n    - Students may progress from a basic explanation of causation by simple reasoning and supporting it with factual knowledge and understanding.\n  example_answers: \n    - Operation Rolling Thunder affected the conflict in Vietnam in the 1960s. This was a bombing campaign by America that targeted government buildings in North Vietnam. The campaign was meant to last for a few weeks but it lasted for several years.\nLevel_1:\n  description: \n    - Basic explanation of one or more factors\n    - Answer demonstrates basic knowledge and understanding that is relevant to the question\n    - Answer demonstrates a basic line of reasoning, which is coherent, structured with some substantiation; the relevance might be implicit.\n  marks_awarded:\n    maximum: 4\n    minimum: 1\n  criteria:\n    - Students recognise and provide a basic explanation of one or more factors.\n  example_answers: \n    - Students may offer a basic explanation of the stated factor, such as North Vietnam attacked a US warship in the Gulf of Tonkin.\n    - Students may offer basic explanations of other factor(s), for example, the US army could not defeat the guerrilla tactics of the Vietcong.\nLevel_0:\n  marks_awarded:\n    maximum: 0\n    minimum: 0\n  criteria:\n    - Students either submit no evidence or fail to address the question",
        "Answer": "I do agree that the Gulf of Tonkin incident was the main reason as to why the conflict in Vietnam escalated. I do agree because the Gulf of Tonkin incident was when America had claimed that Vietnam attacked three of their battle ships. This had led to a right for America that if Vietnam attacked anymore aggression had happened then America attack. Therefore the next day in 1964, or 1965, everything had corrupted, this led to the declaration of war which was why the conflict in Vietnam had escalated in the 1960\u2019s.\nI do not agree that the Gulf of Tonkin was the main reason why the conflict escalated in Vietnam because another event had taken place called Dien Bien Phu. Dien Bien Phu was when the French army had got destroyed so it had meant that in result to this, led to the conflict in Vietnam to escalate in the 1960\u2019s\nI do agree with the statement because the Gulf of Tonkin had led to a Gulf of Tonkin resolution which was in 1964, when after the right of being able to attack, It led to the declaration of war, which was the main reason why the conflict in Vietnam had escalated very quickly.\nI do not agree with this statement because another reason why the conflict in Vietnam escalated in the 1960\u2019s was the Domino theory. This theory was about when if one country had fallen with communism others would go down as well with communism. This had led to a major escalation in Vietnam because the other countries didn\u2019t want to be communist. To conclude this, I agree that the Gulf of Tonkin incident was the main reason as to why the conflict had escalated in Vietnam because it led to the Gulf of Tonkin resolution which was when they had declared war in the 1964 and was the main reason why the Vietnam War had escalated.",
        "Response": "The student addresses the given factor in paragraphs 1 and 3. They show some simple knowledge and explain how that caused escalation of the war in a simple way. They do not show confidence in the knowledge \u2018in 1964 or 1965\u2019, but they do make the link to the question explicit, so achieve level 2. They identify two other factors, Dien Bien Phu and the Domino Theory, but these are not relevant for the question and relate to the 1950s rather than the 1960s. The overall structure is good, but to achieve level 3, the student would need to explain further relevant factors and develop their points further.\nOverall, the answer falls into level 2 and should be given 5 marks."
    },
    {
        "Question": "\u2018The Gulf of Tonkin incident was the main reason why the conflict in Vietnam escalated in the 1960s.\u2019 How far do you agree with this statement? Explain your answer.",
        "Mark Scheme": "Level_4:\n  description: \n    - Complex explanation of stated factor and other factor(s) leading to a sustained judgement\n    - Answer demonstrates a range of accurate and detailed knowledge and understanding that is relevant to the question.\n    - Answer demonstrates a complex, sustained line of reasoning which has a sharply focused coherence and logical structure that is fully substantiated, with well-judged relevance.\n  marks_awarded:\n    maximum: 16\n    minimum: 13\n  progression: \n    - Extends Level 3.\n    - Students may progress from a developed explanation of causation by complex explanation of the relationship between causes supported by detailed factual knowledge and understanding to form a sustained judgement.\n  example_answers: \n    - The Gulf of Tonkin incident was just one of the reasons for escalation in the 1960s. The wider context was that as the war continued, the nature of the conflict made decisive victory impossible. The size and strength of US military power did not bring a great advantage in jungle warfare against the guerrilla tactics of the Vietcong, nor could it destroy the Ho Chi Minh trail.\nLevel_3:\n  description: \n    - Developed explanation of the stated factor and other factor(s)\n    - Answer demonstrates a range of accurate knowledge and understanding that is relevant to the question\n    - Answer demonstrates a developed, sustained line of reasoning which has coherence and logical structure; it is well substantiated, and with sustained, explicit relevance.\n  marks_awarded:\n    maximum: 12\n    minimum: 9\n  progression: \n    - Extends Level 2.\n    - Answer may suggest that one reason has greater merit.\n    - Students may progress from a simple explanation of causation with developed reasoning supported by factual knowledge and understanding.\n  example_answers: \n    - The Gulf of Tonkin incident involved a torpedo attack on USS Maddox and the report of another imminent attack. Although no further attacks occurred, the suspicion was sufficient for the US government to pass a resolution which gave the President the power to take further military against North Vietnam. This escalated the conflict because President Johnson used this power to launch bombing campaigns against North Vietnam and sent thousands of American soldiers to South Vietnam.\n    - The main reason the conflict continued in the 1960s was because the Vietcong successfully fought a guerrilla war. The Vietcong hung on the belts of the Americans. The Americans struggled to identify the enemy because they didn\u2019t wear uniforms and hid in the jungle and launched ambush attacks. The Americans could not be seen to be losing so they kept sending more forces and used new methods of attack such as Agent Orange to clear the forest.\nLevel_2:\n  description: \n    - Simple explanation of stated factor or other factor(s)\n    - Answer demonstrates specific knowledge and understanding that is relevant to the question\n    - Answer demonstrates a simple, sustained line of reasoning which is coherent, structured, substantiated and explicitly relevant.\n    - Answers arguing a preference for one judgement but with only basic explanation of another view will be marked at this level.\n  marks_awarded:\n    maximum: 8\n    minimum: 5\n  progression: \n    - Students may progress from a basic explanation of causation by simple reasoning and supporting it with factual knowledge and understanding.\n  example_answers: \n    - Operation Rolling Thunder affected the conflict in Vietnam in the 1960s. This was a bombing campaign by America that targeted government buildings in North Vietnam. The campaign was meant to last for a few weeks but it lasted for several years.\nLevel_1:\n  description: \n    - Basic explanation of one or more factors\n    - Answer demonstrates basic knowledge and understanding that is relevant to the question\n    - Answer demonstrates a basic line of reasoning, which is coherent, structured with some substantiation; the relevance might be implicit.\n  marks_awarded:\n    maximum: 4\n    minimum: 1\n  criteria:\n    - Students recognise and provide a basic explanation of one or more factors.\n  example_answers: \n    - Students may offer a basic explanation of the stated factor, such as North Vietnam attacked a US warship in the Gulf of Tonkin.\n    - Students may offer basic explanations of other factor(s), for example, the US army could not defeat the guerrilla tactics of the Vietcong.\nLevel_0:\n  marks_awarded:\n    maximum: 0\n    minimum: 0\n  criteria:\n    - Students either submit no evidence or fail to address the question",
        "Answer": "I agree with this statement that one of the main reasons that conflict escalated in Vietnam was because of the Gulf of Tonkin incident. The Gulf of Tonkin incident is where one of the USA ships, the Maddox, was attacked with missiles. American officials were told that this was an attack from North Vietnam which led to the US getting more involved in the war, from this point LBJ set up operation Rolling Thunder, a bombing campaign against North Vietnam. The use of chemical weapons and bombing lead to many North Vietnamese people livelihoods had been destroyed encouraging them to fight back.\nHowever, I do think that there are other incidents that led to the increase in conflict such as Americas search and destroy tactics. This tactic was where Americans troops would search, raid, destroy and burn villages looking for Viet Cong members, this was difficult however because the Viet Cong didn\u2019t were uniforms, so the US troops were killing innocent people. This led to many more Vietnamese people going into the jungle and joining the Viet Cong to fight against the Americans.\nFurthermore, I think another incident that increase the conflict was the Tet Offensive and the My Lai Massacre in 1968. The My Lai Massacre was where some soldiers from US troops went to My Lai and slaughtered 400 innocent Vietnamese civilians however American officials tried to keep this secret until 1969. The unfair killings and treatment led to a lot of a ger and further tension in Vietnam, making them more determined to resist and win the war.\nLastly, another reason conflict started to escalate in the 1960\u2019s was when Diem was in charge of South Vietnam. The USA chose Diem to be in charge because they felt like he could easily be manipulated and controlled, however he wasn\u2019t, he ignored the Geneva agreement and rigged votes this meant that the south Vietnamese started to resist him. The Ho-Chi-Minh trail was erected to help supply them with supplies and resistant groups were set up such as the NLF who would fight and kill government officials involved with him. Eventually the US agreed to a coup being organised against him and he was assassinated in 1963.",
        "Response": "The student starts by explaining the given factor, and this is done in a developed way, with developed reasoning of why escalated the war supported by a range of factual knowledge about the Gulf of Tonkin incident and the US response. The warship and the bombing campaign are both correctly named and how it caused escalation is explained \u2018which led to the US getting more involved\u2019; \u2018encouraging them to fight back\u2019.\nThe student also explains other factors, relevant to the 1960s and the escalation of the war \u2013 the search and destroy tactics and the My Lai Massacre. These are both really part of the same second factor \u2013 the US tactics in Vietnam and the way that it increased opposition from the Vietnamese people. There is developed explanation and specific factual knowledge \u2013 for example, about the My Lai massacre. The explanation of a second factor places this answer into level 3.\nA third factor is explained, though Diem was leader from 1955 so this is not really relevant to the question. There is no attempt to make a judgement in a conclusion, so this answer does not reach level 4.\nOverall, the answer falls into level 3 and should be given 10 marks."
    },
    {
        "Question": "\u2018The Gulf of Tonkin incident was the main reason why the conflict in Vietnam escalated in the 1960s.\u2019 How far do you agree with this statement? Explain your answer.",
        "Mark Scheme": "Level_4:\n  description: \n    - Complex explanation of stated factor and other factor(s) leading to a sustained judgement\n    - Answer demonstrates a range of accurate and detailed knowledge and understanding that is relevant to the question.\n    - Answer demonstrates a complex, sustained line of reasoning which has a sharply focused coherence and logical structure that is fully substantiated, with well-judged relevance.\n  marks_awarded:\n    maximum: 16\n    minimum: 13\n  progression: \n    - Extends Level 3.\n    - Students may progress from a developed explanation of causation by complex explanation of the relationship between causes supported by detailed factual knowledge and understanding to form a sustained judgement.\n  example_answers: \n    - The Gulf of Tonkin incident was just one of the reasons for escalation in the 1960s. The wider context was that as the war continued, the nature of the conflict made decisive victory impossible. The size and strength of US military power did not bring a great advantage in jungle warfare against the guerrilla tactics of the Vietcong, nor could it destroy the Ho Chi Minh trail.\nLevel_3:\n  description: \n    - Developed explanation of the stated factor and other factor(s)\n    - Answer demonstrates a range of accurate knowledge and understanding that is relevant to the question\n    - Answer demonstrates a developed, sustained line of reasoning which has coherence and logical structure; it is well substantiated, and with sustained, explicit relevance.\n  marks_awarded:\n    maximum: 12\n    minimum: 9\n  progression: \n    - Extends Level 2.\n    - Answer may suggest that one reason has greater merit.\n    - Students may progress from a simple explanation of causation with developed reasoning supported by factual knowledge and understanding.\n  example_answers: \n    - The Gulf of Tonkin incident involved a torpedo attack on USS Maddox and the report of another imminent attack. Although no further attacks occurred, the suspicion was sufficient for the US government to pass a resolution which gave the President the power to take further military against North Vietnam. This escalated the conflict because President Johnson used this power to launch bombing campaigns against North Vietnam and sent thousands of American soldiers to South Vietnam.\n    - The main reason the conflict continued in the 1960s was because the Vietcong successfully fought a guerrilla war. The Vietcong hung on the belts of the Americans. The Americans struggled to identify the enemy because they didn\u2019t wear uniforms and hid in the jungle and launched ambush attacks. The Americans could not be seen to be losing so they kept sending more forces and used new methods of attack such as Agent Orange to clear the forest.\nLevel_2:\n  description: \n    - Simple explanation of stated factor or other factor(s)\n    - Answer demonstrates specific knowledge and understanding that is relevant to the question\n    - Answer demonstrates a simple, sustained line of reasoning which is coherent, structured, substantiated and explicitly relevant.\n    - Answers arguing a preference for one judgement but with only basic explanation of another view will be marked at this level.\n  marks_awarded:\n    maximum: 8\n    minimum: 5\n  progression: \n    - Students may progress from a basic explanation of causation by simple reasoning and supporting it with factual knowledge and understanding.\n  example_answers: \n    - Operation Rolling Thunder affected the conflict in Vietnam in the 1960s. This was a bombing campaign by America that targeted government buildings in North Vietnam. The campaign was meant to last for a few weeks but it lasted for several years.\nLevel_1:\n  description: \n    - Basic explanation of one or more factors\n    - Answer demonstrates basic knowledge and understanding that is relevant to the question\n    - Answer demonstrates a basic line of reasoning, which is coherent, structured with some substantiation; the relevance might be implicit.\n  marks_awarded:\n    maximum: 4\n    minimum: 1\n  criteria:\n    - Students recognise and provide a basic explanation of one or more factors.\n  example_answers: \n    - Students may offer a basic explanation of the stated factor, such as North Vietnam attacked a US warship in the Gulf of Tonkin.\n    - Students may offer basic explanations of other factor(s), for example, the US army could not defeat the guerrilla tactics of the Vietcong.\nLevel_0:\n  marks_awarded:\n    maximum: 0\n    minimum: 0\n  criteria:\n    - Students either submit no evidence or fail to address the question",
        "Answer": "The Gulf of Tonkin incident was a significant reason for the rising conflict in Vietnam in the 1960s. This was an incident where, on the 4 August 1964, as US destroyer ship, \u2018Maddox\u2019, detected a North Vietnamese attack. However the captain who raised the alarm had doubts and saw his men may have read the sonar wrong, Nevertheless, on 7th August 1964, due to these reports, the government passed the Gulf of Tonkin Resolution giving US President Johnson the power to take any military action necessary to stop the spread of communism and the power of the North Vietnamese in Vietnam. This incident officially started the war allowing combat troops to be sent. Without this incident, combat troops may never have been sent to Vietnam and so it is extremely significant in it\u2019s role as amplifying conflict.\nAnother reason why conflict escalated was due to international American politics in 1960\u2019s. The president from 1963-68, Johnson, maintained an ant-communist stance and supported president Eisenhower, a previous president, in his idea of the domino theory. This was the idea that of one country fell to communism, every country around it would be affecting US trade. Because US hatred for communism was high, Johnson had to appear tough on communism to maintain public support. Additionally, to become president, weapons companies funded presidential campaigns in hope they will primary weapons providers in war. Thus, this theory of the Military Industrial Complex stakes that Johnson may have felt pressured to be involved in the war to make profit for these companies, appeasing big businesses. This internal American politics was an important driving force behind many of the American\u2019s actions in Vietnam.\nConflict was also escalated due to the Viet Cong\u2019s tactics, The Viet Cong were a guerilla force, fighting against the UIS and South Vietnamese army in the south. Their tactics were effective in disorientating the US as they had no uniform, no known headquarters, hid in jungles and tunnels they constructed. They also ambushed Americans accounting for 51% of US deaths and used booby traps like punji traps (sharpened bamboo in the ground covered with leaves). Booby traps accounted for 11% of US casualties and instilled widespread paranoia in the US soldiers leading to low morale. As a result, the American army found it difficult to progress against the Viet Cong and soldiers grew angry, in part leading to events like the My Lai Massacre on 16th March 1968 where US soldiers killed 500 people, mostly innocent women, children and elderly. The US also employed the use of agent orange, a chemical weapon, to attempt to burn the jungle so the Viet Cong would find it harder to hide, which destroyed acres of jungle. And they used operation Rolling Thunder to drop over a million tonnes of bombs on Vietnam. Thus, the Viet Cong\u2019s tactics caused the Americans to act in desperation, killing civilians and destroying the county\u2019s environment, in turn, leading to conflict developing as more Vietnamese civilians turned to the Viet Cong for support, and they were more resilient than ever in their drive for liberation due to the oppressive, violent nature of the Americans and their attempts to outsmart Viet Cong tactics.\nTo conclude, the Gulf of Tonkin incident was an important reason for conflict amplification. However, the Gulf of Tonkin Resolution, which allowed for original conflict to begin, may not have been passed if it was not for internal American politics and US policy of containment which caused them to have destroyer ships in the Gulf in the first place as a threat to the communist north. Accordingly, the Viet Cong\u2019s tactics lead to increased conflict in Vietnam, as the US restored to harsh tactics to devastate the country and lower support for the Viet Cong. This lengthened the period of the war as the Viet Cong did not give up as easily as the US had hoped. However, America would not be using such harsh tactics or be in Vietnam to begin with, again, if it was not for the US negative attitudes to communism, and for politicians\u2019 need to show that they will be harsh against communism leading to their use of chemical warfare in retaliation and mass bombing campaigns.",
        "Response": "The student addresses the given factor in the first paragraph and supports their reasoning with detailed factual knowledge about the incident and the US response. There is complex evaluation of the importance of this factor \u2018without this incident, combat troops may never have been sent to Vietnam.\u2019\nThe second paragraph shows complex explanation of a second factor and detailed factual knowledge and understanding of the US political situation at the time. The impact of the politics on the Vietnam War is explained, with different aspects considered, showing a high level of complexity. There is some evaluation of the importance of this \u2018an important driving force behind many of the American\u2019s actions in Vietnam\u2019.\nThe third paragraph explains the tactics of the Viet Cong, again supported by factual knowledge and understanding and developed explanation of the US response to these tactics is shown. Specific factual details about the VietCong tactics and the My Lai Massacre elevate the answer and there is explicit focus on the question \u2018leading to conflict developing\u2026\u2019. With three factors explained in a developed way, this answer clearly sits in level 3.\nThe conclusion then secures level 4. Each of the factors are evaluated and compared and the complex relationship between them is explained, to lead to a sustained and substantiated judgement. The conclusion creates an argument with phrases such as \u2018may not have been passed if it was not for\u2026\u2019, \u2018this lengthened the period of the war\u2019, \u2018if it was not for the US negative attitudes\u2026\u2019.\nOverall, this answer falls into level 4 and should be awarded 16 marks."
    },
    {
        "Question": "The Iran-Iraq War was the main reason for tension in the Gulf in 1990.\u2019 How far do you agree with this statement? Explain your answer",
        "Mark Scheme": "Level_4:\n  description: \n    - Complex explanation of stated factor and other factor(s) leading to a sustained judgement\n    - Answer demonstrates a range of accurate and detailed knowledge and understanding that is relevant to the question\n    - Answer demonstrates a complex, sustained line of reasoning which has a sharply-focused coherence and logical structure that is fully substantiated, with well-judged relevance.\n  marks_awarded:\n    maximum: 16\n    minimum: 13\n  criteria:\n    - Extends Level 3.\n    - Students may progress from a developed explanation of causation by complex explanation of the relationship between causes supported by detailed factual knowledge and understanding to form a sustained judgement.\n  example_answers: \n    - The Iran-Iraq war was a symptom rather than the cause of tensions. The war reflected the religious split between hostile Sunni and Shia states; both sides were led by regimes which had ambitions for status and power given the Iranian Revolution and Saddam\u2019s arrogant and aggressive personality. Further instability was caused by Western interests, and the demand for oil. US behaviour fuelled suspicion across the Arab world, supporting Saddam and Israel at the same time. The war reflected long term tensions which explain its length and bitterness.\nLevel_3:\n  description: \n    - Developed explanation of the stated factor and other factor(s)\n    - Answer demonstrates a range of accurate knowledge and understanding that is relevant to the question\n    - Answer demonstrates a developed, sustained line of reasoning which has coherence and logical structure; it is well substantiated, and with sustained, explicit relevance.\n  marks_awarded:\n    maximum: 12\n    minimum: 9\n  criteria:\n    - Extends Level 2.\n    - Answer may suggest that one reason has greater merit.\n    - Students may progress from a simple explanation of causation with developed reasoning supported by factual knowledge and understanding.\n  example_answers: \n    - There was tension in the Gulf because, following the war, areas of dispute had not been settled such as shared use of the Shatt al-Arab waterway, which both countries needed for exports for oil by sea. And when Khomeini died in June 1989, his son vowed to continue the Islamic Revolution and hostility towards Iraq.\n    - There were other main reasons for tension; there were underlying religious differences between Sunnis and Shias which were worsened by the interference of the West in the region. The West\u2019s need for oil and continuing support for Israel against hostile Arab neighbours were bound to cause instability.\nLevel_2:\n  description: \n    - Simple explanation of stated factor or other factor(s)\n    - Answer demonstrates specific knowledge and understanding that is relevant to the question\n    - Answer demonstrates a simple, sustained line of reasoning which is coherent, structured, substantiated and explicitly relevant.\n  marks_awarded:\n    maximum: 8\n    minimum: 5\n  criteria:\n    - Answers arguing a preference for one judgement but with only basic explanation of another view will be marked at this level.\n    - Students may progress from a basic explanation of causation by simple reasoning and supporting it with factual knowledge and understanding.\n  example_answers: \n    - Saddam had survived, and he remained an important threat to Gulf stability because he was rearming to reassert his power and status in the region. Dealing with Iraqi debt might mean aggression against other oil rich states.\nLevel_1:\n  description: \n    - Basic explanation of one or more factors\n    - Answer demonstrates basic knowledge and understanding that is relevant to the question\n    - Answer demonstrates a basic line of reasoning, which is coherent, structured with some substantiation; the relevance might be implicit.\n  marks_awarded:\n    maximum: 4\n    minimum: 1\n  criteria:\n    - Students recognise and provide a basic explanation of one or more factors.\n  example_answers: \n    - Students may offer a basic explanation of the stated factor, such as Iran and Iraq would remain enemies because of religious differences between Sunnis and Shias.\n    - Students may offer basic explanations of other factor(s), for example, western powers were concerned about oil supplies.\nLevel_0:\n  description: \n    - Students either submit no evidence or fail to address the question\n  marks_awarded:\n    maximum: 0\n    minimum: 0",
        "Answer": "The Iran Iraq War began in 1980 with an Iraq offensive on Iranian forces, this was because of trade, land and religious sectarian disputes between the two nations. However also because of the Iranian revolution in 1998 which was supported by Iranian leader Ayatollah Khomeini. He wanted the Shia belief to spread across the Arab world so he could have a larger theocracy. This revolution led to a threat of uprising for Saddam Hussein who was the Iraqi leader and a Sunni Muslim. Iraq was a largely Shia population as well. So, to prevent an attack on Iraq, Saddam decided to strike first on Iranian forces to show strength and unite his nation against a common cause.\nThe war lasted 8 years (1980-88) and many conflicts and disputes were inconclusive on resolving dilemmas over trade, oil and land control. Iraq suffered 5 hundred thousand casualties as a result of the war, and also was left in a debt of $8 billion to several counties across the world. Iraq also had trade and military sanctions placed on them by the UN, meaning other countries were reluctant to trade and import goods to Iraq. This left Iraq in a state of bankruptcy an extreme poverty. Because not only was Iraq deprived of goods, but also not allowed to extract large amounts of oil had. This resulted losing more than 50% of it\u2019s income. Therefore this created tension in the gulf because Iraq were in turmoil and needed a way out which posed a threat to neighboring countries such as Kuwait who could not defend and Iraq invasion to steal resources like oil.\nIran suffered 1 million casualties and were on the verge of bankruptcy after 8 years of fighting, this meant that their country was now in deep poverty and dismay. Furthermore, there was unrest in the country because of the difficulty period and lack of progress from the war, meaning there could have been an uprising at any moment. Many Iranian and Iraqi lives were lost over the Shatt-Al-Arab waterway (a thin strip of trade route bordering Iran and Iraq) yet the control over the area was still disputed by the two countries creating tension in the gulf even after 8 years of fighting.",
        "Response": "The student shows detailed factual knowledge about the Iran-Iraq War and the consequences of this. They address the question directly in paragraph 2 \u2018this created tension in the gulf because\u2026\u2019. There is a good range of factual knowledge and understanding shown throughout. However, the answer is capped at level 2 as the student only addresses the given factor and doesn\u2019t explain any other causes of tension in their answer. To reach level 3, the student must address the stated factor and another factor.\nOverall, the answer falls into level 2 and should be awarded 8 marks."
    },
    {
        "Question": "The Iran-Iraq War was the main reason for tension in the Gulf in 1990.\u2019 How far do you agree with this statement? Explain your answer",
        "Mark Scheme": "Level_4:\n  description: \n    - Complex explanation of stated factor and other factor(s) leading to a sustained judgement\n    - Answer demonstrates a range of accurate and detailed knowledge and understanding that is relevant to the question\n    - Answer demonstrates a complex, sustained line of reasoning which has a sharply-focused coherence and logical structure that is fully substantiated, with well-judged relevance.\n  marks_awarded:\n    maximum: 16\n    minimum: 13\n  criteria:\n    - Extends Level 3.\n    - Students may progress from a developed explanation of causation by complex explanation of the relationship between causes supported by detailed factual knowledge and understanding to form a sustained judgement.\n  example_answers: \n    - The Iran-Iraq war was a symptom rather than the cause of tensions. The war reflected the religious split between hostile Sunni and Shia states; both sides were led by regimes which had ambitions for status and power given the Iranian Revolution and Saddam\u2019s arrogant and aggressive personality. Further instability was caused by Western interests, and the demand for oil. US behaviour fuelled suspicion across the Arab world, supporting Saddam and Israel at the same time. The war reflected long term tensions which explain its length and bitterness.\nLevel_3:\n  description: \n    - Developed explanation of the stated factor and other factor(s)\n    - Answer demonstrates a range of accurate knowledge and understanding that is relevant to the question\n    - Answer demonstrates a developed, sustained line of reasoning which has coherence and logical structure; it is well substantiated, and with sustained, explicit relevance.\n  marks_awarded:\n    maximum: 12\n    minimum: 9\n  criteria:\n    - Extends Level 2.\n    - Answer may suggest that one reason has greater merit.\n    - Students may progress from a simple explanation of causation with developed reasoning supported by factual knowledge and understanding.\n  example_answers: \n    - There was tension in the Gulf because, following the war, areas of dispute had not been settled such as shared use of the Shatt al-Arab waterway, which both countries needed for exports for oil by sea. And when Khomeini died in June 1989, his son vowed to continue the Islamic Revolution and hostility towards Iraq.\n    - There were other main reasons for tension; there were underlying religious differences between Sunnis and Shias which were worsened by the interference of the West in the region. The West\u2019s need for oil and continuing support for Israel against hostile Arab neighbours were bound to cause instability.\nLevel_2:\n  description: \n    - Simple explanation of stated factor or other factor(s)\n    - Answer demonstrates specific knowledge and understanding that is relevant to the question\n    - Answer demonstrates a simple, sustained line of reasoning which is coherent, structured, substantiated and explicitly relevant.\n  marks_awarded:\n    maximum: 8\n    minimum: 5\n  criteria:\n    - Answers arguing a preference for one judgement but with only basic explanation of another view will be marked at this level.\n    - Students may progress from a basic explanation of causation by simple reasoning and supporting it with factual knowledge and understanding.\n  example_answers: \n    - Saddam had survived, and he remained an important threat to Gulf stability because he was rearming to reassert his power and status in the region. Dealing with Iraqi debt might mean aggression against other oil rich states.\nLevel_1:\n  description: \n    - Basic explanation of one or more factors\n    - Answer demonstrates basic knowledge and understanding that is relevant to the question\n    - Answer demonstrates a basic line of reasoning, which is coherent, structured with some substantiation; the relevance might be implicit.\n  marks_awarded:\n    maximum: 4\n    minimum: 1\n  criteria:\n    - Students recognise and provide a basic explanation of one or more factors.\n  example_answers: \n    - Students may offer a basic explanation of the stated factor, such as Iran and Iraq would remain enemies because of religious differences between Sunnis and Shias.\n    - Students may offer basic explanations of other factor(s), for example, western powers were concerned about oil supplies.\nLevel_0:\n  description: \n    - Students either submit no evidence or fail to address the question\n  marks_awarded:\n    maximum: 0\n    minimum: 0",
        "Answer": "I strongly agree with this statement as everything after the Iran-Iraq war was a knock-on effect from Saddam\u2019s failure. His economy was ruined by the war so he tried to get Kuwait to increase their oil prices so that he could then do the same, allowing for Iraq\u2019s economy to rebuild. Their refusal was what caused Saddam to annex them, causing the 1990 Gulf War. This war then caused religious massacres which caused a huge amount of tension among religious groups, many of whom were also angry at the West after their failure to kill or capture Saddam.\nFurthermore, many nations were worried that Saddam would target them, particularly Saudi Arabia as they were world leaders in the Oil market. All neighboring countries were scared and with Iraq being landlocked, surrounded by nations whose economics were comprised of Oil trading, it led to unprecedented tension, Things only worsened after the 9/11 attacks, orchestrated by Osama Bin laden who had previously supported America. His change in attitude towards west was likely due to Saddam not being punished over the war and genocides.\nOsama then evaded US troops for a decade, causing America to ignite a second Gulf War which used a forged piece of evidence on Saddam breaking the peace treaty as a basis for the invasion. This angered Gulf citizens as the invasion was without justification. Even after Saddam was killed, America and other allied nations continued to occupy middle eastern nations, leading to constant battle and many defeats. Gulf citizens blamed the wets due to the usage of fake evidence for the occupation.\nIn conclusion, the Iran-Iraq war led to a two-decade war, the largest terrorist attacks ever seen, multiple genocides, invasions and occupations and the biggest political scandal of the 21st century. The fact that all these atrocities stemmed from the embers of the Iran-Iraq war leads me to conclude that it was responsible for an overwhelming amount of tension in the Gulf.",
        "Response": "The student clearly explains the impact of the Iran-Iraq War and supports this with factual knowledge and understanding. There is a clear attempt to make a judgement at the start \u2018everything after the Iran-Iraq War was a knock-on effect from Saddam\u2019s failure.\u2019 The religious and economic consequences are explained.\nThe second paragraph explains how other countries feared being attacked, particularly due to tensions over the oil market. This is a brief explanation of another factor and so places the answer into level 3.\nThe discussion of 9/11 and its consequences, although well explained, is not relevant to the question, which specifies 1990. Therefore the second half of paragraph 2 and paragraph 3 are not credited. There is a clear judgement made in the conclusion.\nOverall, there is one developed explanation of the stated factor, plus a second factor briefly explained so the answer falls into level 3 and should be awarded 9 marks."
    },
    {
        "Question": "The Iran-Iraq War was the main reason for tension in the Gulf in 1990.\u2019 How far do you agree with this statement? Explain your answer",
        "Mark Scheme": "Level_4:\n  description: \n    - Complex explanation of stated factor and other factor(s) leading to a sustained judgement\n    - Answer demonstrates a range of accurate and detailed knowledge and understanding that is relevant to the question\n    - Answer demonstrates a complex, sustained line of reasoning which has a sharply-focused coherence and logical structure that is fully substantiated, with well-judged relevance.\n  marks_awarded:\n    maximum: 16\n    minimum: 13\n  criteria:\n    - Extends Level 3.\n    - Students may progress from a developed explanation of causation by complex explanation of the relationship between causes supported by detailed factual knowledge and understanding to form a sustained judgement.\n  example_answers: \n    - The Iran-Iraq war was a symptom rather than the cause of tensions. The war reflected the religious split between hostile Sunni and Shia states; both sides were led by regimes which had ambitions for status and power given the Iranian Revolution and Saddam\u2019s arrogant and aggressive personality. Further instability was caused by Western interests, and the demand for oil. US behaviour fuelled suspicion across the Arab world, supporting Saddam and Israel at the same time. The war reflected long term tensions which explain its length and bitterness.\nLevel_3:\n  description: \n    - Developed explanation of the stated factor and other factor(s)\n    - Answer demonstrates a range of accurate knowledge and understanding that is relevant to the question\n    - Answer demonstrates a developed, sustained line of reasoning which has coherence and logical structure; it is well substantiated, and with sustained, explicit relevance.\n  marks_awarded:\n    maximum: 12\n    minimum: 9\n  criteria:\n    - Extends Level 2.\n    - Answer may suggest that one reason has greater merit.\n    - Students may progress from a simple explanation of causation with developed reasoning supported by factual knowledge and understanding.\n  example_answers: \n    - There was tension in the Gulf because, following the war, areas of dispute had not been settled such as shared use of the Shatt al-Arab waterway, which both countries needed for exports for oil by sea. And when Khomeini died in June 1989, his son vowed to continue the Islamic Revolution and hostility towards Iraq.\n    - There were other main reasons for tension; there were underlying religious differences between Sunnis and Shias which were worsened by the interference of the West in the region. The West\u2019s need for oil and continuing support for Israel against hostile Arab neighbours were bound to cause instability.\nLevel_2:\n  description: \n    - Simple explanation of stated factor or other factor(s)\n    - Answer demonstrates specific knowledge and understanding that is relevant to the question\n    - Answer demonstrates a simple, sustained line of reasoning which is coherent, structured, substantiated and explicitly relevant.\n  marks_awarded:\n    maximum: 8\n    minimum: 5\n  criteria:\n    - Answers arguing a preference for one judgement but with only basic explanation of another view will be marked at this level.\n    - Students may progress from a basic explanation of causation by simple reasoning and supporting it with factual knowledge and understanding.\n  example_answers: \n    - Saddam had survived, and he remained an important threat to Gulf stability because he was rearming to reassert his power and status in the region. Dealing with Iraqi debt might mean aggression against other oil rich states.\nLevel_1:\n  description: \n    - Basic explanation of one or more factors\n    - Answer demonstrates basic knowledge and understanding that is relevant to the question\n    - Answer demonstrates a basic line of reasoning, which is coherent, structured with some substantiation; the relevance might be implicit.\n  marks_awarded:\n    maximum: 4\n    minimum: 1\n  criteria:\n    - Students recognise and provide a basic explanation of one or more factors.\n  example_answers: \n    - Students may offer a basic explanation of the stated factor, such as Iran and Iraq would remain enemies because of religious differences between Sunnis and Shias.\n    - Students may offer basic explanations of other factor(s), for example, western powers were concerned about oil supplies.\nLevel_0:\n  description: \n    - Students either submit no evidence or fail to address the question\n  marks_awarded:\n    maximum: 0\n    minimum: 0",
        "Answer": "I agree to a certain extent that the Iran-Iraq War was the main reason for tension in the Gulf in 1990, although a very important reason, it is a combination of multiple factors such as the Israeli Palestine conflict, Saddam Hussein wanting to develop a larger international reputation and American troops being stationed in Saudi Arabia.\nOne reason for the tension in the Gulf was the consequences of the Iran-Iraq War from 1980- 1988. The war ended in a stalemate which means that neither side gained anything. Both sides lost millions of dollars of money and casualties. The issue hadn\u2019t been resolved which means the tension between the two was now at an all time high. The Iran-Iraq war also split the Arab countries public opinion with some countries supporting Iraq (US + western countries) and some supporting Iran (Pakistan). The US also played a big part is the Iran-Iraq War with supplying weapons to Iraq (publicly) and to Iran (privately) This means that both countries were weaker so America could maintain its global hold on the Middle East and its oil.\nAlthough the Iran-Iraq War was a more important reason the continued Israeli-Palestine conflict was still dividing the Middle East\u2019s public opinion. After being establish in the mid 1900\u2019s its existence was born challenged territorially (by Palestine because some of Palestines land was taken away) and religiously because Israel is a Jewish country. There has been constant Israeli-Palestine conflict since its founding which means different countries in the Middle East have different opinions. Israel is supported by US + Western countries and Palestine being supported by Iraq + Pakistan. This tension was escalated by the continued unwanted influence of the US troops and government.\nThe main reason for tension in the Gulf in 1990 was Saddam Hussein (leader of Iraq) wanted to extend his global territory and wanted a better international relationship because he was seen as weak internationally after the Iran-Iraq War. This means that Saddam aimed to invade Kuwait. This was the pinnacle of tension in the Gulf. There was a UN coalition formed and Operation Desert Storm and Operation Desert Shield commenced in 1991. Saddam aimed to get rid of his debt and claimed that Kuwait was historically part of Iraq. This meant that another war had been broken out in an already fractured and crumbled Middle East. This was where the most tension came from.\nIn conclusion the main reason for tension and conflict in the Gulf in 1990 was Saddam\u2019s global policy of wanting to extend his power and reach and boost his reputation after the bad outcome of the Iran-Iraq war from 1980-1988. The Israeli-Palestine conflict contributed to this and America\u2019s constant influence on the region only heightened the already high tensions.",
        "Response": "This is a well-structured answer with a sustained judgement, as the student sets out their argument in a brief introduction and evaluates each factor at the end of each paragraph. The first paragraph explains the stated factor. There is a good range of accurate knowledge and understanding shown and a clear explanation of how this increased tension in 1990.\nThe second paragraph explains a second factor \u2013 \u2018the Israeli-Palestine conflict\u2019 \u2013 again showing a good range of factual knowledge and clearly explaining the ways that tension was increased. The knowledge could be developed further here with more specific detail. The third paragraph explains a third factor -  Saddam Hussein\u2019s desire to extend his global territory \u2013 and the invasion of Kuwait. The importance of this is clearly explained again, supported by factual knowledge. The judgement is sustained as the paragraph begins \u2018the main reason for tension in the Gulf was..\u2019, building the argument throughout the essay.\nThe conclusion repeats earlier points and clarifies the judgement made, making links between the factors. Overall, this answer falls into level 4 and should be awarded 15 marks."
    }
]

In [ ]:
# Load in Data
completed_paper_df = pd.read_csv(data_dir / "student_answers_augmented.csv")
student_answers_records = completed_paper_df.to_dict(orient="records")

In [ ]:
student_answer = [i for i in student_answers_records if (i['student_id'] == 5001 and i['question_id'] == 4.1)][0].copy()
student_answer

# OpenAI API: Learn from Error Nudge

In [ ]:
import openai

# list models
models = openai.Model.list()
[i["id"] for i in models["data"]]

In [ ]:
# Create Prompt
prompt_template = construct_prompt_template(
        instruction=instruction, 
        few_shot_examples=judgement_few_shot_examples).strip()

prompt = prompt_template.format(
    question=student_answer["question_text"],
    mark_scheme=student_answer["mark_scheme_text"],
    answer=student_answer["answer_text"]
)

In [ ]:
# create a chat completion
chat_completion = openai.ChatCompletion.create(
    model="gpt-4o", 
    messages=[
        {"role": "user", "content": prompt}
        ])

# print the chat completion
first_response = chat_completion.choices[0].message.content
print(first_response)

In [ ]:
# Incoporate feedback
feedback = """
Check if your response makes the following error:
**Error**
- Mistaking points used to illustrate a point as separate explanatory points. An answer may touch on multiple points as part of the chain of reasoning detailing one point but they are not separate explanatory points. 
Think step by step and justify your analysis
"""

chat_completion = openai.ChatCompletion.create(
    model="gpt-4o", 
    messages=[
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": first_response},
        {"role": "user", "content": feedback}
        ])

# print the chat completion
feedback_remark = chat_completion.choices[0].message.content
print(feedback_remark)

In [ ]:
chat_completion = openai.ChatCompletion.create(
    model="gpt-4o", 
    messages=[
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": first_response},
        {"role": "user", "content": feedback},
        {"role": "assistant", "content": feedback_remark},
        {"role": "user", "content": "What are the other factors this answer touches upon? I fail to see how this falls into level 3. I believe it should be awarded 0 marks."}
        ])

# print the chat completion
print(chat_completion.choices[0].message.content)

# Naive Majority Vote Approach

In [ ]:
# Initialise T prompt templates with m randomly sampled few shot examples
T = 8
m = 3

prompt_template_dicts = []

for t in range(T):
    seed_num = 456*t
    
    random.seed(seed_num)
    judgement_few_shot_examples_subset = random.sample(judgement_few_shot_examples, m)

    prompt_template = construct_prompt_template(
        instruction=instruction, 
        few_shot_examples=judgement_few_shot_examples_subset).strip()

    GRADE_ANSWER_PROMPT = PromptTemplate.from_template(
        prompt_template
    )

    prompt_template_dicts.append( {"prompt_template": GRADE_ANSWER_PROMPT, "random_seed": seed_num, "few_shot_examples": judgement_few_shot_examples_subset} )

In [ ]:
# Feed all T prompt templates, the prompt and obtain graded marks
responses_lst = []

for _, prompt_template_dict in enumerate(prompt_template_dicts):

    prompt_template = prompt_template_dict["prompt_template"]
    input_args = {
        "question": student_answer["question_text"], 
        "answer": student_answer["answer_text"], 
        "mark_scheme": student_answer["mark_scheme_text"], 
        "context": student_answer["context"]
        }
    prompt_template_input_args = dict((k, input_args[k]) for k in prompt_template.input_variables if k in input_args)

    graded_answer_response = run_prompt_chain(
        prompt_template=prompt_template.template,
        llm=llm,
        burn_in_runs=1,
        **prompt_template_input_args
        )

    graded_answer = graded_answer_response["prompt_chain_response"]
    graded_answer_token_cost = graded_answer_response["prompt_chain_token_costing"]["total_cost"]

    extract_marks_response = run_prompt_chain(
        prompt_template="Extract the number of marks awarded to the student answer. Read the answer carefully and extract the final number of marks to be awarded to the student. Report the number of marks only. Answer: {answer}",
        llm=ChatOpenAI(model_name="gpt-4o", temperature=0.0, max_tokens=500),
        burn_in_runs=1,
        answer=graded_answer
        )

    marks = extract_marks_response["prompt_chain_response"]
    extract_marks_token_cost = extract_marks_response["prompt_chain_token_costing"]["total_cost"]

    output_dict = {"graded_answer": graded_answer, "marks_awarded": marks, "total_cost": graded_answer_token_cost + extract_marks_token_cost}

    responses_lst.append( output_dict )


In [ ]:
# Obtain final marks via majority Vote
from collections import Counter
 
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]

marks_array = [int(i["marks_awarded"]) for i in responses_lst]

print(marks_array)
most_frequent(marks_array)


In [ ]:
costs_array = [float(i["total_cost"]) for i in responses_lst]
print(costs_array)
np.sum(costs_array)

# Pairwise Comparison

1. Arrange $\{r_t, g_t\}_{t=1}^{T}$ into random pairs
2. Use a **comparison** prompt c to compare pairs. c takes a pair of $\{r_t, g_t\}$ as input and returns a winning pair $\{r_t^*, g_t^*\}$
3. Repeat step 3 for multiple rounds until 2 pairs remain - called finalists
4. **Synthesis** the pair into a final reasoning, grade pair $\{r_{t}^{'}, g_{t}^{'}\}$

In [ ]:
import json
import itertools

def extract_json_from_markdown(markdown_string):
    # Remove the triple backticks and 'json' from the input string
    json_string = markdown_string.strip().strip('```json').strip('```').strip()
    
    # Parse the cleaned JSON string into a Python dictionary
    data = json.loads(json_string)

    if isinstance(data, list):
        return data
    else: 
        return [data]

def create_random_pairs(my_list, seed=123):
    """Creates random pairs without replacement from a list.

    Args:
    my_list: The list of elements.

    Returns:
    A list of tuples, where each tuple is a random pair.
    """
    random.seed(seed)
    random.shuffle(my_list)
    pairs = []
    for i in range(0, len(my_list), 2):
        pairs.append((my_list[i], my_list[i+1]))
    
    return pairs

In [ ]:
grading_comparison_raw_prompt = """
You are a senior examiner that reviews the grading and marks awarded to a student answer by more junior examiners.
You are given 2 examiner commentaries, the question, mark scheme and student answer.
Determine which of the examiner commentaries grades the student answer better using the mark scheme as a guide. 
Analyse the examiner commentaries thoroughly and ensure that all claims made are factual, fair and well evidenced from the mark scheme. 
Refer back to commentaries and mark schemes when appropriate.
Return the number of examiner commentary that is preferred.
Do not invent anything.

Question:
{question}

Answer:
{answer}

Mark Scheme:
{mark_scheme}

Examiner Commentary 1
{examiner_commentary1}

Examiner Commentary 2
{examiner_commentary2}

Obey the following JSON format:
'explanation': str,
'preferred_commentary_number': int


Response:
""".strip()

grading_comparison_prompt = PromptTemplate.from_template(
        grading_comparison_raw_prompt
)

In [ ]:
from typing import Dict, List

def comparison_round(
        comparison_pairs: List[Dict], 
        student_answer: Dict[str, str],
        seed: int = 123
        ) -> List[Dict]:
    
    winning_examples_lst = []
    for _, grading_pair in enumerate(comparison_pairs):

        prompt_template_input_args = {
        "question": student_answer["question_text"],
        "answer": student_answer["answer_text"],
        "mark_scheme": student_answer["mark_scheme_text"],
        "examiner_commentary1": grading_pair[0]['graded_answer'],
        "examiner_commentary2": grading_pair[1]["graded_answer"]
        }

        pairwise_comparison = run_prompt_chain(
            prompt_template=grading_comparison_prompt.template,
            llm=llm,
            burn_in_runs=1,
            **prompt_template_input_args
            )
        
        try:
            preferred_commentary = extract_json_from_markdown(pairwise_comparison["prompt_chain_response"])[0]
        except JSONDecodeError as e:
            preferred_commentary = literal_eval(pairwise_comparison["prompt_chain_response"])
        
        preferred_commentary_idx = preferred_commentary["preferred_commentary_number"]
        winning_examples_lst.append( grading_pair[preferred_commentary_idx - 1] )

    if len(winning_examples_lst) == 1:
        winning_pairs = winning_examples_lst[0].copy()
        winning_pairs["comparison_explanation"] = preferred_commentary['explanation']
    else:
        winning_pairs = create_random_pairs(winning_examples_lst, seed=seed)
    
    return winning_pairs


In [ ]:
# Pairwise Comparison of Responses
## Number of Pairs must be a power of 2.
import numpy as np

num_responses = len(responses_lst)
assert np.ceil(np.log2(num_responses)) == np.log2(num_responses), "Number of Responses must be a nonzero power of 2"

## Organise randomly into pairs
seed = 123
refined_responses_lst = [{k: v for k, v in i.items() if k in ["graded_answer", "marks_awarded"]} for i in responses_lst]
grading_pairs = create_random_pairs(refined_responses_lst, seed=seed)

print( grading_pairs )

pairs = grading_pairs.copy()

# Perform a pairwise comparison between graded answer pairs
while True:
    pairs = comparison_round(
        comparison_pairs=pairs, 
        student_answer=student_answer,
        seed=seed
        )
    
    if len(pairs) == 1:
        finalists = pairs.copy()
        break
        
# Select the winning response from final 2 responses
winning_graded_response = comparison_round(
    comparison_pairs=finalists, 
    student_answer=student_answer,
    seed=seed
    )

print("Finalists:\n", finalists)
print("Winning Graded Response: \n", winning_graded_response)


In [ ]:
print( winning_graded_response["comparison_explanation"] )

In [ ]:
print( finalists[0][0]["graded_answer"] )
print("-" * 40)
print( finalists[0][1]["graded_answer"] )

# Generate Representative Clusters

Given a set of reasoning and grading pairs 
$$ \{(r_{t}, g_{t})\}_{t=1}^{T} $$

1. Group Examples based on if they received the same marks, yielding clusters $ G_1, G_2, ..., G_D $ where instances in cluster $G_d$ has the same number of marks
2. Synthesize all examples in a cluster $G_d$ and generate a representative group exemplar.
3. Either:
   i. Synthesis all group exemplars into a single reasoning and grading pair $(r_{j}^{*}, g_{j}^{*})$
   ii. Arrange group exemplars into pairwise comparisons and select the best group exemplar after pairwise comparisons

In [ ]:
# Construct Prompt
grading_synthesize_raw_instruction_prompt = """
You are a senior examiner that reviews the grading and marks awarded to a student answer by more junior examiners. Your role is to ensure that the graded answer is as accurate and well referenced as possible.
You are given a list of graded answers, the question, mark scheme and student answer.
Analyse the graded answers and grade the student answer in light of the information provided. 
Don't be afraid to contradict the graded answers if you find their reasoning chain to be erroneous. 
Ensure that all claims made in the final graded answer are factual, fair and well evidenced from the mark scheme. 
Make sure your answer is brief. Do not invent anything.

Question:
{question}

Answer:
{answer}

Mark Scheme:
{mark_scheme}
""".strip()

In [ ]:
def wrap_single_example(example: Dict) -> str:
    expanded_examples_lst = []
    for key, value in example.items():
        expanded_examples_lst.append(f"{key}:\n{value}")

    return "\n\n".join(expanded_examples_lst)

def wrap_multiple_examples(few_shot_examples: List[Dict]) -> str:

    expanded_examples_lst = []
    for _, example in enumerate(few_shot_examples):
        expanded_examples_lst.append(wrap_single_example(example))

    return "\n\n".join(expanded_examples_lst)

def synthesize_responses(
        student_answer: Dict[str, str],
        responses: List[Dict[str, str]],
        prompt_prefix: str,
        llm: ChatOpenAI = llm
        ) -> str:
    
    examiner_commentaries = wrap_multiple_examples(
        few_shot_examples=responses
        )

    grading_synthesize_raw_instruction_with_answers_prompt = "\n\n".join(
        [prompt_prefix, 
         examiner_commentaries, 
         "Response:"]
         )

    grading_synthesize_prompt = PromptTemplate.from_template(
            grading_synthesize_raw_instruction_with_answers_prompt
    )

    prompt_template_input_args = {
    "question": student_answer["question_text"],
    "answer": student_answer["answer_text"],
    "mark_scheme": student_answer["mark_scheme_text"],
    }

    graded_answer_synthesized = run_prompt_chain(
            prompt_template=grading_synthesize_prompt.template,
            llm=llm,
            burn_in_runs=1,
            **prompt_template_input_args
            )
    
    return graded_answer_synthesized

In [ ]:
# Group Examples
unique_marks = list( set([int(i['marks_awarded']) for i in responses_lst]) )
print( unique_marks )

clusters = {}
for idx, mark in enumerate(unique_marks):
    cluster_idx = f"cluster_{idx}"
    clusters[cluster_idx] = [{k: v for k, v in item.items() if k in ["graded_answer"]}  for item in responses_lst if int( item['marks_awarded'] ) == mark] 

print( clusters )

In [ ]:
# Generate cluster representative graded answers
clusters_with_examplars = {}
for cluster_id, responses in clusters.items():

    num_responses = len(responses)
    print(f"{num_responses} Cluster Members")

    if num_responses == 1:
        clusters_with_examplars[cluster_id] = responses
    else:
        synthesized_graded_answer = synthesize_responses(
            student_answer=student_answer, 
            responses = responses,
            prompt_prefix = grading_synthesize_raw_instruction_prompt
            )
        clusters_with_examplars[cluster_id] = [ {"graded_answer": synthesized_graded_answer["prompt_chain_response"] } ]
    

cluster_exemplars = [item for sublist in clusters_with_examplars.values() for item in sublist]
synthesized_graded_answer = synthesize_responses(
    student_answer=student_answer, 
    responses = cluster_exemplars,
    prompt_prefix = grading_synthesize_raw_instruction_prompt
    )

In [ ]:
print(synthesized_graded_answer["prompt_chain_response"])

# Review and Combine Prompting

In [ ]:
review_prompt = """
You are an expert examiner who identifies weaknesses in student answers given a mark scheme. 
You are meticulous and clearly identify when a student answer loses marks according to the Mark Scheme.

Question 
{question}

Mark Scheme
{mark_scheme}

Answer
{answer}

Mention where the student loses marks and how many marks they've lost if any.
Be as harsh in your grading.
It is imperative that you are accurate in your analysis. Explain your reasoning.
"""

combine_prompt = """
Question 
{question}

Mark Scheme
{mark_scheme}

Answer
{answer}

Graded Answer 1
{answer1}

Graded Answer 2
{answer2}

Rewrite Graded Answer 1 given the information provided by Graded Answer 2. 
Resolve any discrepancies with reference to the Answer and Mark Scheme. 
Recall that the Mark Scheme rewards other relevant answers, outside of the examples, be sensitive to this.
Quote the Answer as much as possible.
Keep the writing structure of Graded Answer 1, any changes should be minimal.
The final line of your response should be how many marks in total the answer has been awarded.
"""

review_answer_prompt = PromptTemplate.from_template(
        review_prompt
)

combine_answer_prompt = PromptTemplate.from_template(
    combine_prompt
)

In [ ]:
original_graded_answer = responses_lst[1]["graded_answer"]

In [ ]:
# Review and Combine

## Identify weaknesses in awarded grade
review_grade = run_prompt_chain(
    prompt_template = review_answer_prompt.template,
    llm=llm,
    burn_in_runs=1,
    question=student_answer["question_text"],
    mark_scheme=student_answer["mark_scheme_text"],
    answer=student_answer["answer_text"]
)


## Combine grade and review into a final grade
combined_graded_answer = run_prompt_chain(
    prompt_template = combine_answer_prompt.template, 
    llm = llm, 
    burn_in_runs = 1,
    question = student_answer["question_text"],
    mark_scheme = student_answer["mark_scheme_text"],
    answer = student_answer["answer_text"], 
    answer1 = original_graded_answer,
    answer2 = review_grade["prompt_chain_response"]
    )

In [ ]:
print("Review Grade:\n", review_grade["prompt_chain_response"])
print("-" * 40)
print("Combined Grade:\n", combined_graded_answer["prompt_chain_response"])


# Centre of Gravity Approach
Given $L$ marking levels that a given answer can be awarded.

1. For each level, $l_{j}$ in the mark scheme, use the LLM to generate a rationale justifying why the student answer should be placed in the selected marking level $l$.

$$ LLM(\mathbf{x}_{i}, l_{j}) $$

where $l_{j}$ is the j-th marking level in the mark scheme.

2. For graded answers corresponding to marking levels - $a_1, a_2,...,a_L$, either:
    i. Perform successive pairwise comparisons to determine best answer
    ii. Synthesize answers

In [ ]:
marking_level_prompt = """
You are an expert examiner who reviews gradings of student answers. 
You have been provided with a Question, Mark Scheme, Answer and an Awarded Marking Level.
Given the Awarded Marking Level, justify why the student answer achieves this level with evidence from the Mark Scheme and Answer.
Reference the Mark Scheme and Answer as much as possible, ensure you are as thorough and meticulous as possible.

Question 
{question}

Mark Scheme
{mark_scheme}

Answer
{answer}

Awarded Marking Level
{awarded_level}

At the end of your answer, state how many marks the student answer should be awarded.
"""

marking_level_prompt_template = PromptTemplate.from_template(
    marking_level_prompt
)

In [ ]:
marking_levels = [1, 2, 3, 4]

level_grading_lst = []
for idx, marking_level in enumerate(marking_levels):
    
    cog_graded_answer = run_prompt_chain(
    prompt_template = marking_level_prompt_template.template, 
    llm = ChatOpenAI(model="gpt-4o", temperature=0.0, max_tokens=800), 
    burn_in_runs = 1,
    question = student_answer["question_text"],
    mark_scheme = student_answer["mark_scheme_text"],
    answer = student_answer["answer_text"], 
    awarded_level = marking_level,
    )

    level_grading_lst.append( cog_graded_answer )


In [ ]:
cog_responses = [i["prompt_chain_response"] for i in level_grading_lst]
cog_responses_dict = [ dict( zip( ["graded_answer"]*len(marking_levels), cog_responses) ) ]

In [ ]:
cog_responses

In [ ]:
grading_synthesize_raw_instruction_prompt = """
You are a senior examiner that reviews the grading and marks awarded to a student answer by more junior examiners. Your role is to ensure that the graded answer is as accurate and well referenced as possible.
You are given a list of graded answers, the question, mark scheme and student answer.
Analyse the graded answers and grade the student answer in light of the information provided. 
Don't be afraid to contradict the graded answers if you find their reasoning chain to be erroneous. 
Ensure that all claims made in the final graded answer are factual, fair and well evidenced from the mark scheme. 
Make sure your answer is brief. Do not invent anything.

Question:
{question}

Answer:
{answer}

Mark Scheme:
{mark_scheme}
""".strip()

synthesized_level_answer = synthesize_responses(
    student_answer=student_answer, 
    responses = cog_responses_dict,
    prompt_prefix = grading_synthesize_raw_instruction_prompt,
    llm = ChatOpenAI(model="gpt-4o", temperature=0.0, max_tokens=800)
    )

In [ ]:
print( synthesized_level_answer["prompt_chain_response"] )